# Import statements

In [1]:
# Necessary imports
import gymnasium
from gymnasium import spaces
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import base64, io
import numpy as np
from collections import deque, namedtuple

# For visualization
from IPython.display import HTML
from IPython import display 
import glob
import pandas as pd

In [2]:
import stable_baselines3
from stable_baselines3.sac.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import DQN

-------------------------------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------------------------------

# Setting 1

In [3]:
#Function for normal distribution truncation:

from scipy.stats import truncnorm

def get_truncated_normal(mean, sd, low, upp):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

#Function to get observation (still for single env, need to modify for multi-agent env):

def full_obs(_cap_dem_chosen_req, number_of_actions):
    _obs_min_arr = np.full((288+number_of_actions-1, ), 0)
    for i in range(len(_cap_dem_chosen_req)):
        _obs_min_arr[i+int((number_of_actions-1)/2)] = min(_cap_dem_chosen_req[i])
    return _obs_min_arr

#Function to get the one-hot-encoded vectors for departure and arrival airports:

def one_hot_encode_airport(airport, num_airports):
    encoding = np.zeros(num_airports)
    encoding[airport] = 1
    return encoding

# Example usage
num_airports = 2
airport1 = 0
airport2 = 1

encoded_airport1 = one_hot_encode_airport(airport1, num_airports)
encoded_airport2 = one_hot_encode_airport(airport2, num_airports)

print(encoded_airport1)
print(encoded_airport2)

#Generate full info for the arrival sides:

def generate_info_arv(requests):
    ts_arv = np.empty(shape=(len(requests),), dtype='object')
    for i in range(len(requests)):
        ts_arv[i] = requests[i][1] + requests[i][4]/5
        if ts_arv[i] > 287:
            ts_arv[i] = ts_arv[i] - 287
    return ts_arv

#Generate the multi-agent scenario:

#=======================================================

#Modify the distribution based on historical data later:
def generate_scenario(number_of_requests, num_airports, cap_dict):
    
    #number_of_requests = 1500
    ts_72 = get_truncated_normal(mean=72, sd=36, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_72 = np.round(ts_72)

    ts_216 = get_truncated_normal(mean=216, sd=36, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_216 = np.round(ts_216)

    ts_dep = np.concatenate((ts_72, ts_216))
    ts_dep = ts_dep.astype(int)

    #Generate index for requests:

    index = np.array(list(range(number_of_requests)))

    #Generate origin (0 and 1 are two considered origin airports, 2 represent other airports, encoded in one-hot vector):

    #num_airports = 3
    origin_airport = np.empty(shape=(number_of_requests,), dtype='object')
    destination_airport = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range(number_of_requests):
        _org_airport = one_hot_encode_airport(random.randint(0,1), num_airports)
        _org_airport_list = _org_airport.tolist()
        origin_airport[i] = _org_airport_list
        #Generate destination (the destination will be different with the origin):
        _dest_airport = _org_airport.copy()
        while np.array_equal(_dest_airport, _org_airport):
            np.random.shuffle(_dest_airport)
        _dest_airport_list = _dest_airport.tolist()
        destination_airport[i] = _dest_airport_list

    #Generate flying time (assume between airport 0 and 1 is 2 hour, 0 to 2 and 1 to 2 is arbitrary):

    fly_time = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range (number_of_requests):
        if origin_airport[i] == list([1.0, 0.0]) and destination_airport[i] == list([0.0, 1.0]):
            fly_time[i] = 120
        elif origin_airport[i] == list([0.0, 1.0]) and destination_airport[i] == list([1.0, 0.0]):
            fly_time[i] = random.choice([60, 120, 180])

    #Generate status cap:

    status_cap_dep = np.full((number_of_requests,), 0)
    status_cap_arv = np.full((number_of_requests,), 0)
    

    requests = np.stack((index, ts_dep, origin_airport, destination_airport, fly_time, status_cap_dep), axis=1)

    #Generate full info for the arv side:

    ts_arv = generate_info_arv(requests)

    #pseudo_belong_dep = np.full((number_of_requests,), 0)
    #pseudo_belong_arv = np.full((number_of_requests,), 0)
    
    # Define requests_full as dtype object
    num_entries = len(index)  # Given that 'index' is defined using np.array(list(range(number_of_requests)))
    # Create an empty array of the desired shape with dtype=object
    requests_full = np.empty((num_entries, 8), dtype=object)
    # Fill the array
    data = [index, ts_dep, origin_airport, destination_airport, fly_time, status_cap_dep, ts_arv, status_cap_arv]
    for i, column_data in enumerate(data):
        requests_full[:, i] = column_data

    # airport_req_dict: A dictionary where each key corresponds to a specific airport's requirements. 
    # The key format is 'req_i', where i is the index of the airport. 
    # The value for each key is a numpy array, with each row representing a request and the columns containing different attributes of that request.
    airport_req_dict, _belong_airport_dict = get_airport_req_dict(requests_full, num_airports)

    pot_dem_dict = get_initial_pot_dem_per_airport(airport_req_dict, num_airports)

    cap_dem_dict = get_cap_dem_dict(num_airports, cap_dict, pot_dem_dict)

    return requests_full, airport_req_dict, _belong_airport_dict, pot_dem_dict, cap_dem_dict

def flatten_cap_dem_dict(cap_dem_dict, num_airports):
    cap_dem_dict_flat = {}
    for i in range(num_airports):
        cap_dem_dict_flat['req_{}'.format(i)] = cap_dem_dict['req_{}'.format(i)].flatten()
    return cap_dem_dict_flat

#Get separated req per airport and store in a dict:

def get_airport_req_dict(requests_full, num_airports):
    airport_req_dict = {}
    _belong_airport_dict = {}
    for i in range(num_airports):
        airport_req_dict['req_{}'.format(i)] = np.empty((0, 10)) #This one depends on the number of elements of a final request
        _belong_airport_dict['req_{}'.format(i)] = np.full(num_airports, 0.0, dtype=float)
        _belong_airport_dict['req_{}'.format(i)][i] = float(1.0)
        _belong_airport_dict['req_{}'.format(i)] = _belong_airport_dict['req_{}'.format(i)].tolist()
        
    for i in range(len(requests_full)):
        _found_dep = 0
        _found_arv = 0
        for k in range(num_airports):
            #_found_dep = 0
            #_found_arv = 0
            if requests_full[i][2] == _belong_airport_dict['req_{}'.format(k)]:
                _dep_req = np.append(requests_full[i], 1)
                _dep_req = np.append(_dep_req, 0)
                airport_req_dict['req_{}'.format(k)] = np.vstack((airport_req_dict['req_{}'.format(k)], _dep_req))
                _found_dep = 1

            if requests_full[i][3] == _belong_airport_dict['req_{}'.format(k)]:
                _arv_req = np.append(requests_full[i], 0)
                _arv_req = np.append(_arv_req, 1)
                airport_req_dict['req_{}'.format(k)] = np.vstack((airport_req_dict['req_{}'.format(k)], _arv_req))
                _found_arv = 1

            if _found_dep + _found_arv == 2:
                break
        if _found_dep + _found_arv != 2:
            print("Cannot found both dep and arv at req {}".format(i))
    
    return airport_req_dict, _belong_airport_dict

def generate_deterministic_capacity_dict(num_airports, cap_per_airport_arr): #This function is for 288 slots/ day
    cap_dict = {}
    for i in range(num_airports):
        cap_dict['req_{}'.format(i)] = np.full((288), cap_per_airport_arr[i])
    return cap_dict

def get_date_plot(timeslot_movements, i):
    indices, items = zip(*timeslot_movements)
    sum_of_values = sum(item[1] for item in timeslot_movements)
    print("no of request: ", sum_of_values)
    plt.figure(figsize=(10,6))
    plt.bar(indices, items, width=3.0) 
 
    plt.xlabel('Timeslot') 
    plt.ylabel('No. of movements') 

    plt.title('No. of movements by timeslot in airport' +  str(i)) 

    plt.show()

def get_initial_pot_dem_per_airport(airport_req_dict, num_airports): #Replace req_df to req_df_update to update pot_dem_df #To be replaced with final_sched
    pot_dem_dict = {}
    #TODO: increase speed
    #8 dep 9 arv, 1 dep ts, 6 arv ts
    for i in range(num_airports):
        pot_dem_dict['req_{}'.format(i)] = np.full((288), 0)
        for k in range(len(airport_req_dict['req_{}'.format(i)])):
            _time_slot = int(airport_req_dict['req_{}'.format(i)][k][1]) * int(airport_req_dict['req_{}'.format(i)][k][8]) + int(airport_req_dict['req_{}'.format(i)][k][6]) * int(airport_req_dict['req_{}'.format(i)][k][9])
            pot_dem_dict['req_{}'.format(i)][_time_slot] += 1

    # for i in range(num_airports):
    #     indexed_array = [[index, item] for index, item in enumerate(pot_dem_dict['req_{}'.format(i)])]
    #     get_date_plot(indexed_array, i)
        
    return pot_dem_dict

def get_cap_dem_dict(num_airports, cap_dict, pot_dem_dict):
    cap_dem_dict = {}
    for i in range(num_airports):
        cap_dem_dict['req_{}'.format(i)] = cap_dict['req_{}'.format(i)] - pot_dem_dict['req_{}'.format(i)]
    return cap_dem_dict

[1. 0.]
[0. 1.]


In [4]:
class TwoAirportSchedEnv(gymnasium.Env):
    def __init__(self, number_of_actions, number_of_requests, num_airports, cap_per_airport_arr):
        super(TwoAirportSchedEnv, self).__init__()
        self.number_of_actions = int(number_of_actions)
        self.number_of_requests = number_of_requests
        self.num_airports = num_airports
        self.cap_per_airport_arr = cap_per_airport_arr
        self.cap_dict = generate_deterministic_capacity_dict(self.num_airports, self.cap_per_airport_arr)
        _exceed_cap = 0         
        while _exceed_cap == 0:
            self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = generate_scenario(number_of_requests = self.number_of_requests, num_airports = self.num_airports, cap_dict = self.cap_dict)
            self.cap_dem_dict_flat = flatten_cap_dem_dict(self.cap_dem_dict, self.num_airports)
            for i in range(num_airports):
                _exceed_cap = _exceed_cap + (min(self.cap_dem_dict_flat['req_{}'.format(i)]))

        # initialize the status capacity of all requests in requests_full
        self.initialize_status_capacity(self.cap_dem_dict, self.requests_full)
        # obtain the violate_set
        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)
        #to choose the request that will be checked
        self.get_req(self.requests_full, self.violate_set)

        #to generate action:
        self.action_space = spaces.Discrete(self.number_of_actions)
        self.observation_space = spaces.Box(low= -np.inf, high= np.inf, shape=(26,), dtype=float)
        self.num_step = 0
        
        self.original_requests = self.requests_full.copy()

    def initialize_status_capacity(self, cap_dem_dict, requests_full):
        for request_index in range(len(requests_full)):
            ts_dep = int(requests_full[request_index, 1])
            ts_arv = int(requests_full[request_index, 6])
            origin_airport = int(requests_full[request_index, 2].index(1.0))
            destination_airport = int(requests_full[request_index, 3].index(1.0))

            cap_dem_value_dep = cap_dem_dict['req_{}'.format(origin_airport)][ts_dep]
            cap_dem_value_arv = cap_dem_dict['req_{}'.format(destination_airport)][ts_arv]
            
            if cap_dem_value_dep >= 0:
                requests_full[request_index, 5] = 0
            if cap_dem_value_dep < 0:
                requests_full[request_index, 5] = 1
            if cap_dem_value_arv >= 0:
                requests_full[request_index, 7] = 0
            if cap_dem_value_arv < 0:
                requests_full[request_index, 7] = 1
    
    def get_violate_id_set(self, cap_dem_dict, requests_full):
        violate_set = []
        
        for request_index in range(len(requests_full)):
            status_cap_dep = requests_full[request_index, 5]
            status_cap_arv = requests_full[request_index, 7]

            if status_cap_dep == 1 or status_cap_arv == 1:
                violate_set.append(request_index)
        
        return violate_set
    
    #added get request to know which is the request we are going to move at this step",
    #Randomly choose a violating request to solve
    def get_req(self, requests_full, violate_set):
        if violate_set:
            chosen_request_id = random.choice(violate_set)
        else:
            print("The violate_set is empty. No value to select.")
            chosen_request_id = 0
        
        self.chosen_req = self.requests_full[self.requests_full[:,0] == chosen_request_id][0]

        return self.chosen_req
    
    def action_mapping(self, action, number_of_actions):
        mid_point = (number_of_actions - 1) // 2
        return action - mid_point
    
    #need to check arrival time or change the variable to check_outbound(arv) and chec_outbound(dep)
    def check_outbound(self, action):
        dep_time_slot = self.chosen_req[1]
        arv_time_slot = self.chosen_req[6]

        outbound = False

        new_dep_time_slot = dep_time_slot + action
        new_arv_time_slot = arv_time_slot + action

        if (new_dep_time_slot < 0) or (new_dep_time_slot > 287) or (new_arv_time_slot < 0) or (new_arv_time_slot > 287):
            outbound = True

        return outbound
    
    def dep_or_arv(self):
        pass

    def update_dem(self, dep_airport, arv_airport, time_slot_dep, time_slot_arv, new_time_slot_dep, new_time_slot_arv):
        # Increment demand for the new time slot for both departure and arrival airports
        self.pot_dem_dict['req_{}'.format(dep_airport)][new_time_slot_dep] += 1
        self.pot_dem_dict['req_{}'.format(arv_airport)][new_time_slot_arv] += 1

        # Decrease demand for the initial time slot for both departure and arrival airports
        self.pot_dem_dict['req_{}'.format(dep_airport)][time_slot_dep] -= 1
        self.pot_dem_dict['req_{}'.format(arv_airport)][time_slot_arv] -= 1

    def update_cap_dem(self):
        for airport in range(self.num_airports):
            cap_key = 'req_{}'.format(airport)
            dem_key = 'req_{}'.format(airport)
            if cap_key not in self.cap_dict:
                raise KeyError(f"'{cap_key}' not found in cap_dict. Available keys: {list(self.cap_dict.keys())}")
            self.cap_dem_dict[cap_key] = self.cap_dict[cap_key] - self.pot_dem_dict[dem_key]

    def get_airport_index(self, airport_type):
        # Check for airport type
        if airport_type == "departure":
            airport_index = self.chosen_req[2].index(1.0)
        elif airport_type == "arrival":
            airport_index = self.chosen_req[3].index(1.0)
        else:
            raise ValueError(f"Invalid airport_type: {airport_type}. Expected 'departure' or 'arrival'.")

        return airport_index
    
    def when_take_action(self, action):   
        _index = self.chosen_req[0]
        time_slot_dep = self.chosen_req[1]
        time_slot_arv = self.chosen_req[6]
        time_slot_arv = int(time_slot_arv)
        _dep_airport = self.chosen_req[2].index(1.0)
        _arv_airport = self.chosen_req[3].index(1.0)
        new_time_slot_dep = time_slot_dep + action
        new_time_slot_arv = time_slot_arv + action
        new_time_slot_arv = int(new_time_slot_arv)

        outbound = self.check_outbound(action)

        if not outbound:
            pass
            
        elif outbound:
            # new_time_slot_dep = random.randint(0, 287)
            # new_time_slot_arv = random.randint(0, 287)
            new_time_slot_dep = time_slot_dep
            new_time_slot_arv = time_slot_arv

        else:
            print('Problem with check outbound!')

        # Update the chosen request for both departure and arrival time slots
        self.chosen_req[1] = new_time_slot_dep
        self.chosen_req[6] = new_time_slot_arv

        # Update demand for the new time slots
        self.update_dem(_dep_airport, _arv_airport, time_slot_dep, time_slot_arv, new_time_slot_dep, new_time_slot_arv)

        # Update cap_dem
        self.update_cap_dem()
        
        if self.cap_dem_dict['req_{}'.format(_dep_airport)][new_time_slot_dep] >= 0:
            self.chosen_req[5] = 0
        else:
            self.chosen_req[5] = 1

        if self.cap_dem_dict['req_{}'.format(_arv_airport)][new_time_slot_arv] >= 0:
            self.chosen_req[7] = 0
        else:
            self.chosen_req[7] = 1
        
        self.requests_full[self.requests_full[:, 0] == _index] = self.chosen_req

        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)

    def step(self, action): 
        num_actions = self.number_of_actions
        # # Convert the discrete action to your desired action value
        real_action = self.action_mapping(action, num_actions)

        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        
        # observation concats the following:
        # ts_dep + cap_dem_dep + ts_arv + cap_dem_arv
        self.when_take_action(real_action)
        outbound = self.check_outbound(real_action)

        # Count number of remaining unaccommodated requests
        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        
        # Reward part:
        done = False
        self.num_step += 1
        
        if outbound:
            real_action = 0
            
        if real_action != 0:
            local_reward = 0.1*(-abs(real_action))
            
            if self.chosen_req[5] == 0 and self.chosen_req[7] == 0:
                reward_solved = 1

            else:
                reward_solved = 0

            if (_unaccom_req == 0 or len(self.violate_set) == 0) and self.num_step != self.number_of_requests*5:
                done = True
                self.num_step = 0
                obs = np.zeros(26)
                global_reward = 30

            elif self.num_step == self.number_of_requests*5:
                done = True
                self.num_step = 0
                obs = np.zeros(26)
                global_reward = _unaccom_req * (-1)

            else:
                global_reward = 0
                obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)
            reward_time_step = -0.8
        
        else:
            reward_time_step = -0.8
            local_reward = 0
            global_reward = 0
            reward_solved = 0
            obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)

        total_reward = float(local_reward + reward_solved + global_reward + reward_time_step)

        truncated = False
    
        return obs, total_reward, done, truncated, {}

    def _next_observation(self, airport_req_dict, num_airports, cap_dem_dict, requests_full):

        # Check if violate_set is empty before getting a request
        if len(self.violate_set) == 0:
            # self.chosen_req = None
            # Return a default observation or handle as necessary
            return np.zeros(26)
        
        # To choose the request that will be checked
        self.get_req(self.requests_full, self.violate_set)
        _ts_dep = int(self.chosen_req[1])  # Ensure _ts_dep is an integer
        _ts_arv = int(self.chosen_req[6])  # Ensure _ts_arv is an integer
        _dep_airport = self.chosen_req[2].index(1.0)
        _arv_airport = self.chosen_req[3].index(1.0)
        cap_dem_arr_dep = self.cap_dem_dict['req_{}'.format(_dep_airport)]
        cap_dem_arr_arv = self.cap_dem_dict['req_{}'.format(_arv_airport)]
    
        # Function to safely get the time slot range with edge case handling
        def get_time_slots(array, time_slot):
            # Initialize an array of -100s
            time_slots = np.full(13, -100.0)
    
            start = max(0, time_slot - 6)
            end = min(len(array), time_slot + 7)  # +7 because the end index is exclusive
            actual_slots = array[start:end]
    
            # Calculate the offset for start and end
            start_offset = 6 - (time_slot - start)
            end_offset = 6 + (end - time_slot)
    
            # Fill the actual values from the array into the initialized array
            time_slots[start_offset:start_offset + len(actual_slots)] = actual_slots
    
            return time_slots
    
        # Getting the relevant time slots for departure and arrival
        relevant_dep = get_time_slots(cap_dem_arr_dep, _ts_dep)
        relevant_arv = get_time_slots(cap_dem_arr_arv, _ts_arv)
    
        # Explicitly declare self.obs as a fixed-size array of length 26
        self.obs = np.empty(26)
        self.obs[:13] = relevant_dep
        self.obs[13:] = relevant_arv
        
        return self.obs

    def reset(self, seed=None):                
        self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = generate_scenario(self.number_of_requests, self.num_airports, cap_dict = self.cap_dict)
        self.original_requests = self.requests_full.copy()
        cap_dem_arr_airport1 = self.cap_dem_dict['req_{}'.format(0)]
        cap_dem_arr_airport2 = self.cap_dem_dict['req_{}'.format(1)]
        
        _cap_dem_flat_airport1 = cap_dem_arr_airport1.flatten()
        _cap_dem_flat_airport2 = cap_dem_arr_airport2.flatten()

        while (min(_cap_dem_flat_airport1) >= 0 or min(_cap_dem_flat_airport2) >= 0):
            self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = generate_scenario(self.number_of_requests, self.num_airports, cap_dict = self.cap_dict)
            _cap_dem_flat_airport1 = cap_dem_arr_airport1.flatten()
            _cap_dem_flat_airport2 = cap_dem_arr_airport2.flatten()
        
        # initialize the status capacity of all requests in requests_full
        self.initialize_status_capacity(self.cap_dem_dict, self.requests_full)
        # obtain the violate_set
        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)
        
        # print('Number of violation for airport1: ', len(_cap_dem_flat_airport1[_cap_dem_flat_airport1 < 0]))
        # print('Number of violation for airport2: ', len(_cap_dem_flat_airport2[_cap_dem_flat_airport2 < 0]))

        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        print("Number of unaccomodated requests at the start of current episode:", _unaccom_req)

        self.num_step = 0
        info = {}
        return self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full), info
    
    # Need to validate
    def evaluate_metrics(self):
        _initial_cap_dem_dict = {}
        _initial_pot_dem_dict = get_initial_pot_dem_per_airport(self.airport_req_dict, self.num_airports)
        for airport in range(self.num_airports):
            cap_key = 'req_{}'.format(airport)
            dem_key = 'req_{}'.format(airport)
            _initial_cap_dem_dict[cap_key] = self.cap_dict[cap_key] - _initial_pot_dem_dict[dem_key]

        # Code to print initial number of violation requests for each airport agent
        _initial_cap_dem_airport1 = _initial_cap_dem_dict['req_{}'.format(0)]
        _initial_cap_dem_flat_airport1 = _initial_cap_dem_airport1.flatten()
        _initial_violate_airport1 = len(_initial_cap_dem_flat_airport1[_initial_cap_dem_flat_airport1 < 0])
        print('Initial count of unit violation for airport1 is: ', _initial_violate_airport1)
        _initial_cap_dem_airport2 = _initial_cap_dem_dict['req_{}'.format(1)]
        _initial_cap_dem_flat_airport2 = _initial_cap_dem_airport2.flatten()
        _initial_violate_airport2 = len(_initial_cap_dem_flat_airport2[_initial_cap_dem_flat_airport2 < 0])
        print('Initial count of unit violation for airport2 is: ', _initial_violate_airport2)

        # Code to print final number of violation requests for each airport agent
        cap_dem_arr_airport1 = self.cap_dem_dict['req_{}'.format(0)]
        cap_dem_arr_airport2 = self.cap_dem_dict['req_{}'.format(1)]
        _final_cap_dem_flat_airport1 = cap_dem_arr_airport1.flatten()
        _final_cap_dem_flat_airport2 = cap_dem_arr_airport2.flatten()
        _final_violate_airport1 = len(_final_cap_dem_flat_airport1[_final_cap_dem_flat_airport1 < 0])
        _final_violate_airport2 = len(_final_cap_dem_flat_airport2[_final_cap_dem_flat_airport2 < 0])
        print('Final count of unit violation for airport1 is: ', _final_violate_airport1)
        print('Final count of unit violation for airport2 is: ', _final_violate_airport2)

        _total_sched_delay = sum(abs(self.original_requests[:,1] - self.requests_full[:,1]))  
        print('Total schedule delay is: ', _total_sched_delay)

        _max_shift = max(abs(self.original_requests[:,1] - self.requests_full[:,1]))
        print('Max shift: ', _max_shift)

        # Code to print number of unaccommodated requests at the end of the episode
        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        print('Number of unaccommodated requests: ', _unaccom_req)

        return _initial_violate_airport1, _initial_violate_airport2, _final_violate_airport1, _final_violate_airport2, _total_sched_delay, _max_shift, _unaccom_req

    def set_scenario(self, scenario):
        # Extract components from the scenario tuple
        self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = scenario
        self.original_requests = self.requests_full.copy()

        # Initialize the status capacity of all requests in requests_full
        self.initialize_status_capacity(self.cap_dem_dict, self.requests_full)

        # Obtain the violate_set
        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)

        # Reset step count
        self.num_step = 0

        # Generate the initial observation
        obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)

        self._unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])

        # Return the initial observation and additional info (if any)
        return obs, {}

-------------------------------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------------------------------

# Setting 2

In [7]:
#Function for normal distribution truncation:

from scipy.stats import truncnorm

def get_truncated_normal(mean, sd, low, upp):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

#Function to get observation (still for single env, need to modify for multi-agent env):

def full_obs(_cap_dem_chosen_req, number_of_actions):
    _obs_min_arr = np.full((288+number_of_actions-1, ), 0)
    for i in range(len(_cap_dem_chosen_req)):
        _obs_min_arr[i+int((number_of_actions-1)/2)] = min(_cap_dem_chosen_req[i])
    return _obs_min_arr

#Function to get the one-hot-encoded vectors for departure and arrival airports:

def one_hot_encode_airport(airport, num_airports):
    encoding = np.zeros(num_airports)
    encoding[airport] = 1
    return encoding

# Example usage
num_airports = 2
airport1 = 0
airport2 = 1

encoded_airport1 = one_hot_encode_airport(airport1, num_airports)
encoded_airport2 = one_hot_encode_airport(airport2, num_airports)

print(encoded_airport1)
print(encoded_airport2)

#Generate full info for the arrival sides:

def generate_info_arv(requests):
    ts_arv = np.empty(shape=(len(requests),), dtype='object')
    for i in range(len(requests)):
        ts_arv[i] = requests[i][1] + requests[i][4]/5
        if ts_arv[i] > 287:
            ts_arv[i] = ts_arv[i] - 287
    return ts_arv

#Generate the multi-agent scenario:

#=======================================================

#Modify the distribution based on historical data later:
def generate_scenario(number_of_requests, num_airports, cap_dict):
    
    #number_of_requests = 1500
    ts_72 = get_truncated_normal(mean=72, sd=60, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_72 = np.round(ts_72)

    ts_216 = get_truncated_normal(mean=216, sd=60, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_216 = np.round(ts_216)

    ts_dep = np.concatenate((ts_72, ts_216))
    ts_dep = ts_dep.astype(int)

    #Generate index for requests:

    index = np.array(list(range(number_of_requests)))

    #Generate origin (0 and 1 are two considered origin airports, 2 represent other airports, encoded in one-hot vector):

    #num_airports = 3
    origin_airport = np.empty(shape=(number_of_requests,), dtype='object')
    destination_airport = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range(number_of_requests):
        _org_airport = one_hot_encode_airport(random.randint(0,1), num_airports)
        _org_airport_list = _org_airport.tolist()
        origin_airport[i] = _org_airport_list
        #Generate destination (the destination will be different with the origin):
        _dest_airport = _org_airport.copy()
        while np.array_equal(_dest_airport, _org_airport):
            np.random.shuffle(_dest_airport)
        _dest_airport_list = _dest_airport.tolist()
        destination_airport[i] = _dest_airport_list

    #Generate flying time (assume between airport 0 and 1 is 2 hour, 0 to 2 and 1 to 2 is arbitrary):

    fly_time = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range (number_of_requests):
        if origin_airport[i] == list([1.0, 0.0]) and destination_airport[i] == list([0.0, 1.0]):
            fly_time[i] = 120
        elif origin_airport[i] == list([0.0, 1.0]) and destination_airport[i] == list([1.0, 0.0]):
            fly_time[i] = random.choice([60, 120, 180])

    #Generate status cap:

    status_cap_dep = np.full((number_of_requests,), 0)
    status_cap_arv = np.full((number_of_requests,), 0)
    

    requests = np.stack((index, ts_dep, origin_airport, destination_airport, fly_time, status_cap_dep), axis=1)

    #Generate full info for the arv side:

    ts_arv = generate_info_arv(requests)

    #pseudo_belong_dep = np.full((number_of_requests,), 0)
    #pseudo_belong_arv = np.full((number_of_requests,), 0)
    
    # Define requests_full as dtype object
    num_entries = len(index)  # Given that 'index' is defined using np.array(list(range(number_of_requests)))
    # Create an empty array of the desired shape with dtype=object
    requests_full = np.empty((num_entries, 8), dtype=object)
    # Fill the array
    data = [index, ts_dep, origin_airport, destination_airport, fly_time, status_cap_dep, ts_arv, status_cap_arv]
    for i, column_data in enumerate(data):
        requests_full[:, i] = column_data

    # airport_req_dict: A dictionary where each key corresponds to a specific airport's requirements. 
    # The key format is 'req_i', where i is the index of the airport. 
    # The value for each key is a numpy array, with each row representing a request and the columns containing different attributes of that request.
    airport_req_dict, _belong_airport_dict = get_airport_req_dict(requests_full, num_airports)

    pot_dem_dict = get_initial_pot_dem_per_airport(airport_req_dict, num_airports)

    cap_dem_dict = get_cap_dem_dict(num_airports, cap_dict, pot_dem_dict)

    return requests_full, airport_req_dict, _belong_airport_dict, pot_dem_dict, cap_dem_dict

def flatten_cap_dem_dict(cap_dem_dict, num_airports):
    cap_dem_dict_flat = {}
    for i in range(num_airports):
        cap_dem_dict_flat['req_{}'.format(i)] = cap_dem_dict['req_{}'.format(i)].flatten()
    return cap_dem_dict_flat

#Get separated req per airport and store in a dict:

def get_airport_req_dict(requests_full, num_airports):
    airport_req_dict = {}
    _belong_airport_dict = {}
    for i in range(num_airports):
        airport_req_dict['req_{}'.format(i)] = np.empty((0, 10)) #This one depends on the number of elements of a final request
        _belong_airport_dict['req_{}'.format(i)] = np.full(num_airports, 0.0, dtype=float)
        _belong_airport_dict['req_{}'.format(i)][i] = float(1.0)
        _belong_airport_dict['req_{}'.format(i)] = _belong_airport_dict['req_{}'.format(i)].tolist()
        
    for i in range(len(requests_full)):
        _found_dep = 0
        _found_arv = 0
        for k in range(num_airports):
            #_found_dep = 0
            #_found_arv = 0
            if requests_full[i][2] == _belong_airport_dict['req_{}'.format(k)]:
                _dep_req = np.append(requests_full[i], 1)
                _dep_req = np.append(_dep_req, 0)
                airport_req_dict['req_{}'.format(k)] = np.vstack((airport_req_dict['req_{}'.format(k)], _dep_req))
                _found_dep = 1

            if requests_full[i][3] == _belong_airport_dict['req_{}'.format(k)]:
                _arv_req = np.append(requests_full[i], 0)
                _arv_req = np.append(_arv_req, 1)
                airport_req_dict['req_{}'.format(k)] = np.vstack((airport_req_dict['req_{}'.format(k)], _arv_req))
                _found_arv = 1

            if _found_dep + _found_arv == 2:
                break
        if _found_dep + _found_arv != 2:
            print("Cannot found both dep and arv at req {}".format(i))
    
    return airport_req_dict, _belong_airport_dict

def generate_deterministic_capacity_dict(num_airports, cap_per_airport_arr): #This function is for 288 slots/ day
    cap_dict = {}
    for i in range(num_airports):
        cap_dict['req_{}'.format(i)] = np.full((288), cap_per_airport_arr[i])
    return cap_dict

def get_date_plot(timeslot_movements, i):
    indices, items = zip(*timeslot_movements)
    sum_of_values = sum(item[1] for item in timeslot_movements)
    print("no of request: ", sum_of_values)
    plt.figure(figsize=(10,6))
    plt.bar(indices, items, width=3.0) 
 
    plt.xlabel('Timeslot') 
    plt.ylabel('No. of movements') 

    plt.title('No. of movements by timeslot in airport' +  str(i)) 

    plt.show()

def get_initial_pot_dem_per_airport(airport_req_dict, num_airports): #Replace req_df to req_df_update to update pot_dem_df #To be replaced with final_sched
    pot_dem_dict = {}
    #TODO: increase speed
    #8 dep 9 arv, 1 dep ts, 6 arv ts
    for i in range(num_airports):
        pot_dem_dict['req_{}'.format(i)] = np.full((288), 0)
        for k in range(len(airport_req_dict['req_{}'.format(i)])):
            _time_slot = int(airport_req_dict['req_{}'.format(i)][k][1]) * int(airport_req_dict['req_{}'.format(i)][k][8]) + int(airport_req_dict['req_{}'.format(i)][k][6]) * int(airport_req_dict['req_{}'.format(i)][k][9])
            pot_dem_dict['req_{}'.format(i)][_time_slot] += 1

    # for i in range(num_airports):
    #     indexed_array = [[index, item] for index, item in enumerate(pot_dem_dict['req_{}'.format(i)])]
    #     get_date_plot(indexed_array, i)
        
    return pot_dem_dict

def get_cap_dem_dict(num_airports, cap_dict, pot_dem_dict):
    cap_dem_dict = {}
    for i in range(num_airports):
        cap_dem_dict['req_{}'.format(i)] = cap_dict['req_{}'.format(i)] - pot_dem_dict['req_{}'.format(i)]
    return cap_dem_dict

[1. 0.]
[0. 1.]


In [7]:
class TwoAirportSchedEnv(gymnasium.Env):
    def __init__(self, number_of_actions, number_of_requests, num_airports, cap_per_airport_arr):
        super(TwoAirportSchedEnv, self).__init__()
        self.number_of_actions = int(number_of_actions)
        self.number_of_requests = number_of_requests
        self.num_airports = num_airports
        self.cap_per_airport_arr = cap_per_airport_arr
        self.cap_dict = generate_deterministic_capacity_dict(self.num_airports, self.cap_per_airport_arr)
        _exceed_cap = 0         
        while _exceed_cap == 0:
            self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = generate_scenario(number_of_requests = self.number_of_requests, num_airports = self.num_airports, cap_dict = self.cap_dict)
            self.cap_dem_dict_flat = flatten_cap_dem_dict(self.cap_dem_dict, self.num_airports)
            for i in range(num_airports):
                _exceed_cap = _exceed_cap + (min(self.cap_dem_dict_flat['req_{}'.format(i)]))

        # initialize the status capacity of all requests in requests_full
        self.initialize_status_capacity(self.cap_dem_dict, self.requests_full)
        # obtain the violate_set
        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)
        #to choose the request that will be checked
        self.get_req(self.requests_full, self.violate_set)

        #to generate action:
        self.action_space = spaces.Discrete(self.number_of_actions)
        self.observation_space = spaces.Box(low= -np.inf, high= np.inf, shape=(26,), dtype=float)
        self.num_step = 0
        
        self.original_requests = self.requests_full

    def initialize_status_capacity(self, cap_dem_dict, requests_full):
        for request_index in range(len(requests_full)):
            ts_dep = int(requests_full[request_index, 1])
            ts_arv = int(requests_full[request_index, 6])
            origin_airport = int(requests_full[request_index, 2].index(1.0))
            destination_airport = int(requests_full[request_index, 3].index(1.0))

            cap_dem_value_dep = cap_dem_dict['req_{}'.format(origin_airport)][ts_dep]
            cap_dem_value_arv = cap_dem_dict['req_{}'.format(destination_airport)][ts_arv]
            
            if cap_dem_value_dep >= 0:
                requests_full[request_index, 5] = 0
            if cap_dem_value_dep < 0:
                requests_full[request_index, 5] = 1
            if cap_dem_value_arv >= 0:
                requests_full[request_index, 7] = 0
            if cap_dem_value_arv < 0:
                requests_full[request_index, 7] = 1
    
    def get_violate_id_set(self, cap_dem_dict, requests_full):
        violate_set = []
        
        for request_index in range(len(requests_full)):
            status_cap_dep = requests_full[request_index, 5]
            status_cap_arv = requests_full[request_index, 7]

            if status_cap_dep == 1 or status_cap_arv == 1:
                violate_set.append(request_index)
        
        return violate_set
    
    #added get request to know which is the request we are going to move at this step",
    #Randomly choose a violating request to solve
    def get_req(self, requests_full, violate_set):
        if violate_set:
            chosen_request_id = random.choice(violate_set)
        else:
            print("The violate_set is empty. No value to select.")
            chosen_request_id = 0
        
        self.chosen_req = self.requests_full[self.requests_full[:,0] == chosen_request_id][0]

        return self.chosen_req
    
    def action_mapping(self, action, number_of_actions):
        mid_point = (number_of_actions - 1) // 2
        return action - mid_point
    
    #need to check arrival time or change the variable to check_outbound(arv) and chec_outbound(dep)
    def check_outbound(self, action):
        dep_time_slot = self.chosen_req[1]
        arv_time_slot = self.chosen_req[6]

        outbound = False

        new_dep_time_slot = dep_time_slot + action
        new_arv_time_slot = arv_time_slot + action

        if (new_dep_time_slot < 0) or (new_dep_time_slot > 287) or (new_arv_time_slot < 0) or (new_arv_time_slot > 287):
            outbound = True

        return outbound
    
    def dep_or_arv(self):
        pass

    def update_dem(self, dep_airport, arv_airport, time_slot_dep, time_slot_arv, new_time_slot_dep, new_time_slot_arv):
        # Increment demand for the new time slot for both departure and arrival airports
        self.pot_dem_dict['req_{}'.format(dep_airport)][new_time_slot_dep] += 1
        self.pot_dem_dict['req_{}'.format(arv_airport)][new_time_slot_arv] += 1

        # Decrease demand for the initial time slot for both departure and arrival airports
        self.pot_dem_dict['req_{}'.format(dep_airport)][time_slot_dep] -= 1
        self.pot_dem_dict['req_{}'.format(arv_airport)][time_slot_arv] -= 1

    def update_cap_dem(self):
        for airport in range(self.num_airports):
            cap_key = 'req_{}'.format(airport)
            dem_key = 'req_{}'.format(airport)
            if cap_key not in self.cap_dict:
                raise KeyError(f"'{cap_key}' not found in cap_dict. Available keys: {list(self.cap_dict.keys())}")
            self.cap_dem_dict[cap_key] = self.cap_dict[cap_key] - self.pot_dem_dict[dem_key]

    def get_airport_index(self, airport_type):
        # Check for airport type
        if airport_type == "departure":
            airport_index = self.chosen_req[2].index(1.0)
        elif airport_type == "arrival":
            airport_index = self.chosen_req[3].index(1.0)
        else:
            raise ValueError(f"Invalid airport_type: {airport_type}. Expected 'departure' or 'arrival'.")

        return airport_index
    
    def when_take_action(self, action):   
        _index = self.chosen_req[0]
        time_slot_dep = self.chosen_req[1]
        time_slot_arv = self.chosen_req[6]
        time_slot_arv = int(time_slot_arv)
        _dep_airport = self.chosen_req[2].index(1.0)
        _arv_airport = self.chosen_req[3].index(1.0)
        new_time_slot_dep = time_slot_dep + action
        new_time_slot_arv = time_slot_arv + action
        new_time_slot_arv = int(new_time_slot_arv)

        outbound = self.check_outbound(action)

        if not outbound:
            pass
            
        elif outbound:
            # new_time_slot_dep = random.randint(0, 287)
            # new_time_slot_arv = random.randint(0, 287)
            new_time_slot_dep = time_slot_dep
            new_time_slot_arv = time_slot_arv

        else:
            print('Problem with check outbound!')

        # Update the chosen request for both departure and arrival time slots
        self.chosen_req[1] = new_time_slot_dep
        self.chosen_req[6] = new_time_slot_arv

        # Update demand for the new time slots
        self.update_dem(_dep_airport, _arv_airport, time_slot_dep, time_slot_arv, new_time_slot_dep, new_time_slot_arv)

        # Update cap_dem
        self.update_cap_dem()
        
        if self.cap_dem_dict['req_{}'.format(_dep_airport)][new_time_slot_dep] >= 0:
            self.chosen_req[5] = 0
        else:
            self.chosen_req[5] = 1

        if self.cap_dem_dict['req_{}'.format(_arv_airport)][new_time_slot_arv] >= 0:
            self.chosen_req[7] = 0
        else:
            self.chosen_req[7] = 1
        
        self.requests_full[self.requests_full[:, 0] == _index] = self.chosen_req

        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)

    def step(self, action): 
        num_actions = self.number_of_actions
        # # Convert the discrete action to your desired action value
        real_action = self.action_mapping(action, num_actions)

        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        
        # observation concats the following:
        # ts_dep + cap_dem_dep + ts_arv + cap_dem_arv
        self.when_take_action(real_action)
        outbound = self.check_outbound(real_action)

        # Count number of remaining unaccommodated requests
        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        
        # Reward part:
        done = False
        self.num_step += 1
        
        if outbound:
            real_action = 0
            
        if real_action != 0:
            local_reward = 0.1*(-abs(real_action))
            
            if self.chosen_req[5] == 0 and self.chosen_req[7] == 0:
                reward_solved = 1

            else:
                reward_solved = 0

            if (_unaccom_req == 0 or len(self.violate_set) == 0) and self.num_step != self.number_of_requests*5:
                done = True
                self.num_step = 0
                obs = np.zeros(26)
                global_reward = 30

            elif self.num_step == self.number_of_requests*5:
                done = True
                self.num_step = 0
                obs = np.zeros(26)
                global_reward = _unaccom_req * (-1)

            else:
                global_reward = 0
                obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)
            reward_time_step = -0.8
        
        else:
            reward_time_step = -0.8
            local_reward = 0
            global_reward = 0
            reward_solved = 0
            obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)

        total_reward = float(local_reward + reward_solved + global_reward + reward_time_step)

        truncated = False
    
        return obs, total_reward, done, truncated, {}

    def _next_observation(self, airport_req_dict, num_airports, cap_dem_dict, requests_full):

        # Check if violate_set is empty before getting a request
        if len(self.violate_set) == 0:
            # self.chosen_req = None
            # Return a default observation or handle as necessary
            return np.zeros(26)
        
        # To choose the request that will be checked
        self.get_req(self.requests_full, self.violate_set)
        _ts_dep = int(self.chosen_req[1])  # Ensure _ts_dep is an integer
        _ts_arv = int(self.chosen_req[6])  # Ensure _ts_arv is an integer
        _dep_airport = self.chosen_req[2].index(1.0)
        _arv_airport = self.chosen_req[3].index(1.0)
        cap_dem_arr_dep = self.cap_dem_dict['req_{}'.format(_dep_airport)]
        cap_dem_arr_arv = self.cap_dem_dict['req_{}'.format(_arv_airport)]
    
        # Function to safely get the time slot range with edge case handling
        def get_time_slots(array, time_slot):
            # Initialize an array of -100s
            time_slots = np.full(13, -100.0)
    
            start = max(0, time_slot - 6)
            end = min(len(array), time_slot + 7)  # +7 because the end index is exclusive
            actual_slots = array[start:end]
    
            # Calculate the offset for start and end
            start_offset = 6 - (time_slot - start)
            end_offset = 6 + (end - time_slot)
    
            # Fill the actual values from the array into the initialized array
            time_slots[start_offset:start_offset + len(actual_slots)] = actual_slots
    
            return time_slots
    
        # Getting the relevant time slots for departure and arrival
        relevant_dep = get_time_slots(cap_dem_arr_dep, _ts_dep)
        relevant_arv = get_time_slots(cap_dem_arr_arv, _ts_arv)
    
        # Explicitly declare self.obs as a fixed-size array of length 26
        self.obs = np.empty(26)
        self.obs[:13] = relevant_dep
        self.obs[13:] = relevant_arv
        
        return self.obs

    def reset(self, seed=None):                
        self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = generate_scenario(self.number_of_requests, self.num_airports, cap_dict = self.cap_dict)
        self.original_requests = self.requests_full.copy()
        cap_dem_arr_airport1 = self.cap_dem_dict['req_{}'.format(0)]
        cap_dem_arr_airport2 = self.cap_dem_dict['req_{}'.format(1)]
        
        _cap_dem_flat_airport1 = cap_dem_arr_airport1.flatten()
        _cap_dem_flat_airport2 = cap_dem_arr_airport2.flatten()

        while (min(_cap_dem_flat_airport1) >= 0 or min(_cap_dem_flat_airport2) >= 0):
            self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = generate_scenario(self.number_of_requests, self.num_airports, cap_dict = self.cap_dict)
            _cap_dem_flat_airport1 = cap_dem_arr_airport1.flatten()
            _cap_dem_flat_airport2 = cap_dem_arr_airport2.flatten()
        
        # initialize the status capacity of all requests in requests_full
        self.initialize_status_capacity(self.cap_dem_dict, self.requests_full)
        # obtain the violate_set
        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)
        
        # print('Number of violation for airport1: ', len(_cap_dem_flat_airport1[_cap_dem_flat_airport1 < 0]))
        # print('Number of violation for airport2: ', len(_cap_dem_flat_airport2[_cap_dem_flat_airport2 < 0]))

        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        print("Number of unaccomodated requests at the start of current episode:", _unaccom_req)

        self.num_step = 0
        info = {}
        return self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full), info
    
    # Need to validate
    def evaluate_metrics(self):
        _initial_cap_dem_dict = {}
        _initial_pot_dem_dict = get_initial_pot_dem_per_airport(self.airport_req_dict, self.num_airports)
        for airport in range(self.num_airports):
            cap_key = 'req_{}'.format(airport)
            dem_key = 'req_{}'.format(airport)
            _initial_cap_dem_dict[cap_key] = self.cap_dict[cap_key] - _initial_pot_dem_dict[dem_key]

        # Code to print initial number of violation requests for each airport agent
        _initial_cap_dem_airport1 = _initial_cap_dem_dict['req_{}'.format(0)]
        _initial_cap_dem_flat_airport1 = _initial_cap_dem_airport1.flatten()
        _initial_violate_airport1 = len(_initial_cap_dem_flat_airport1[_initial_cap_dem_flat_airport1 < 0])
        print('Initial count of unit violation for airport1 is: ', _initial_violate_airport1)
        _initial_cap_dem_airport2 = _initial_cap_dem_dict['req_{}'.format(1)]
        _initial_cap_dem_flat_airport2 = _initial_cap_dem_airport2.flatten()
        _initial_violate_airport2 = len(_initial_cap_dem_flat_airport2[_initial_cap_dem_flat_airport2 < 0])
        print('Initial count of unit violation for airport2 is: ', _initial_violate_airport2)

        # Code to print final number of violation requests for each airport agent
        cap_dem_arr_airport1 = self.cap_dem_dict['req_{}'.format(0)]
        cap_dem_arr_airport2 = self.cap_dem_dict['req_{}'.format(1)]
        _final_cap_dem_flat_airport1 = cap_dem_arr_airport1.flatten()
        _final_cap_dem_flat_airport2 = cap_dem_arr_airport2.flatten()
        _final_violate_airport1 = len(_final_cap_dem_flat_airport1[_final_cap_dem_flat_airport1 < 0])
        _final_violate_airport2 = len(_final_cap_dem_flat_airport2[_final_cap_dem_flat_airport2 < 0])
        print('Final count of unit violation for airport1 is: ', _final_violate_airport1)
        print('Final count of unit violation for airport2 is: ', _final_violate_airport2)

        _total_sched_delay = sum(abs(self.original_requests[:,1] - self.requests_full[:,1]))  
        print('Total schedule delay is: ', _total_sched_delay)

        _max_shift = max(abs(self.original_requests[:,1] - self.requests_full[:,1]))
        print('Max shift: ', _max_shift)

        # Code to print number of unaccommodated requests at the end of the episode
        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        print('Number of unaccommodated requests: ', _unaccom_req)

        return _initial_violate_airport1, _initial_violate_airport2, _final_violate_airport1, _final_violate_airport2, _total_sched_delay, _max_shift, _unaccom_req

    def set_scenario(self, scenario):
        # Extract components from the scenario tuple
        self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = scenario
        self.original_requests = self.requests_full.copy()

        # Initialize the status capacity of all requests in requests_full
        self.initialize_status_capacity(self.cap_dem_dict, self.requests_full)

        # Obtain the violate_set
        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)

        # Reset step count
        self.num_step = 0

        # Generate the initial observation
        obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)

        self._unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])

        # Return the initial observation and additional info (if any)
        return obs, {}

# Setting 3

In [15]:
#Function for normal distribution truncation:

from scipy.stats import truncnorm

def get_truncated_normal(mean, sd, low, upp):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

#Function to get observation (still for single env, need to modify for multi-agent env):

def full_obs(_cap_dem_chosen_req, number_of_actions):
    _obs_min_arr = np.full((288+number_of_actions-1, ), 0)
    for i in range(len(_cap_dem_chosen_req)):
        _obs_min_arr[i+int((number_of_actions-1)/2)] = min(_cap_dem_chosen_req[i])
    return _obs_min_arr

#Function to get the one-hot-encoded vectors for departure and arrival airports:

def one_hot_encode_airport(airport, num_airports):
    encoding = np.zeros(num_airports)
    encoding[airport] = 1
    return encoding

# Example usage
num_airports = 2
airport1 = 0
airport2 = 1

encoded_airport1 = one_hot_encode_airport(airport1, num_airports)
encoded_airport2 = one_hot_encode_airport(airport2, num_airports)

print(encoded_airport1)
print(encoded_airport2)

#Generate full info for the arrival sides:

def generate_info_arv(requests):
    ts_arv = np.empty(shape=(len(requests),), dtype='object')
    for i in range(len(requests)):
        ts_arv[i] = requests[i][1] + requests[i][4]/5
        if ts_arv[i] > 287:
            ts_arv[i] = ts_arv[i] - 287
    return ts_arv

#Generate the multi-agent scenario:

#=======================================================

#Modify the distribution based on historical data later:
def generate_scenario(number_of_requests, num_airports, cap_dict):
    
    #number_of_requests = 1500
    ts_72 = get_truncated_normal(mean=72, sd=36, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_72 = np.round(ts_72)

    ts_216 = get_truncated_normal(mean=216, sd=36, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_216 = np.round(ts_216)

    ts_dep = np.concatenate((ts_72, ts_216))
    ts_dep = ts_dep.astype(int)

    #Generate index for requests:

    index = np.array(list(range(number_of_requests)))

    #Generate origin (0 and 1 are two considered origin airports, 2 represent other airports, encoded in one-hot vector):

    #num_airports = 3
    origin_airport = np.empty(shape=(number_of_requests,), dtype='object')
    destination_airport = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range(number_of_requests):
        _org_airport = one_hot_encode_airport(random.randint(0,1), num_airports)
        _org_airport_list = _org_airport.tolist()
        origin_airport[i] = _org_airport_list
        #Generate destination (the destination will be different with the origin):
        _dest_airport = _org_airport.copy()
        while np.array_equal(_dest_airport, _org_airport):
            np.random.shuffle(_dest_airport)
        _dest_airport_list = _dest_airport.tolist()
        destination_airport[i] = _dest_airport_list

    #Generate flying time (assume between airport 0 and 1 is 2 hour, 0 to 2 and 1 to 2 is arbitrary):

    fly_time = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range (number_of_requests):
        if origin_airport[i] == list([1.0, 0.0]) and destination_airport[i] == list([0.0, 1.0]):
            fly_time[i] = 120
        elif origin_airport[i] == list([0.0, 1.0]) and destination_airport[i] == list([1.0, 0.0]):
            fly_time[i] = random.choice([60, 120, 180])

    #Generate status cap:

    status_cap_dep = np.full((number_of_requests,), 0)
    status_cap_arv = np.full((number_of_requests,), 0)
    

    requests = np.stack((index, ts_dep, origin_airport, destination_airport, fly_time, status_cap_dep), axis=1)

    #Generate full info for the arv side:

    ts_arv = generate_info_arv(requests)

    #pseudo_belong_dep = np.full((number_of_requests,), 0)
    #pseudo_belong_arv = np.full((number_of_requests,), 0)
    
    # Define requests_full as dtype object
    num_entries = len(index)  # Given that 'index' is defined using np.array(list(range(number_of_requests)))
    # Create an empty array of the desired shape with dtype=object
    requests_full = np.empty((num_entries, 8), dtype=object)
    # Fill the array
    data = [index, ts_dep, origin_airport, destination_airport, fly_time, status_cap_dep, ts_arv, status_cap_arv]
    for i, column_data in enumerate(data):
        requests_full[:, i] = column_data

    # airport_req_dict: A dictionary where each key corresponds to a specific airport's requirements. 
    # The key format is 'req_i', where i is the index of the airport. 
    # The value for each key is a numpy array, with each row representing a request and the columns containing different attributes of that request.
    airport_req_dict, _belong_airport_dict = get_airport_req_dict(requests_full, num_airports)

    pot_dem_dict = get_initial_pot_dem_per_airport(airport_req_dict, num_airports)

    cap_dem_dict = get_cap_dem_dict(num_airports, cap_dict, pot_dem_dict)

    return requests_full, airport_req_dict, _belong_airport_dict, pot_dem_dict, cap_dem_dict

def flatten_cap_dem_dict(cap_dem_dict, num_airports):
    cap_dem_dict_flat = {}
    for i in range(num_airports):
        cap_dem_dict_flat['req_{}'.format(i)] = cap_dem_dict['req_{}'.format(i)].flatten()
    return cap_dem_dict_flat

#Get separated req per airport and store in a dict:

def get_airport_req_dict(requests_full, num_airports):
    airport_req_dict = {}
    _belong_airport_dict = {}
    for i in range(num_airports):
        airport_req_dict['req_{}'.format(i)] = np.empty((0, 10)) #This one depends on the number of elements of a final request
        _belong_airport_dict['req_{}'.format(i)] = np.full(num_airports, 0.0, dtype=float)
        _belong_airport_dict['req_{}'.format(i)][i] = float(1.0)
        _belong_airport_dict['req_{}'.format(i)] = _belong_airport_dict['req_{}'.format(i)].tolist()
        
    for i in range(len(requests_full)):
        _found_dep = 0
        _found_arv = 0
        for k in range(num_airports):
            #_found_dep = 0
            #_found_arv = 0
            if requests_full[i][2] == _belong_airport_dict['req_{}'.format(k)]:
                _dep_req = np.append(requests_full[i], 1)
                _dep_req = np.append(_dep_req, 0)
                airport_req_dict['req_{}'.format(k)] = np.vstack((airport_req_dict['req_{}'.format(k)], _dep_req))
                _found_dep = 1

            if requests_full[i][3] == _belong_airport_dict['req_{}'.format(k)]:
                _arv_req = np.append(requests_full[i], 0)
                _arv_req = np.append(_arv_req, 1)
                airport_req_dict['req_{}'.format(k)] = np.vstack((airport_req_dict['req_{}'.format(k)], _arv_req))
                _found_arv = 1

            if _found_dep + _found_arv == 2:
                break
        if _found_dep + _found_arv != 2:
            print("Cannot found both dep and arv at req {}".format(i))
    
    return airport_req_dict, _belong_airport_dict

def generate_deterministic_capacity_dict(num_airports, cap_per_airport_arr): #This function is for 288 slots/ day
    cap_dict = {}
    for i in range(num_airports):
        cap_dict['req_{}'.format(i)] = np.full((288), cap_per_airport_arr[i])
    return cap_dict

def get_date_plot(timeslot_movements, i):
    indices, items = zip(*timeslot_movements)
    sum_of_values = sum(item[1] for item in timeslot_movements)
    print("no of request: ", sum_of_values)
    plt.figure(figsize=(10,6))
    plt.bar(indices, items, width=3.0) 
 
    plt.xlabel('Timeslot') 
    plt.ylabel('No. of movements') 

    plt.title('No. of movements by timeslot in airport' +  str(i)) 

    plt.show()

def get_initial_pot_dem_per_airport(airport_req_dict, num_airports): #Replace req_df to req_df_update to update pot_dem_df #To be replaced with final_sched
    pot_dem_dict = {}
    #TODO: increase speed
    #8 dep 9 arv, 1 dep ts, 6 arv ts
    for i in range(num_airports):
        pot_dem_dict['req_{}'.format(i)] = np.full((288), 0)
        for k in range(len(airport_req_dict['req_{}'.format(i)])):
            _time_slot = int(airport_req_dict['req_{}'.format(i)][k][1]) * int(airport_req_dict['req_{}'.format(i)][k][8]) + int(airport_req_dict['req_{}'.format(i)][k][6]) * int(airport_req_dict['req_{}'.format(i)][k][9])
            pot_dem_dict['req_{}'.format(i)][_time_slot] += 1

    # for i in range(num_airports):
    #     indexed_array = [[index, item] for index, item in enumerate(pot_dem_dict['req_{}'.format(i)])]
    #     get_date_plot(indexed_array, i)
        
    return pot_dem_dict

def get_cap_dem_dict(num_airports, cap_dict, pot_dem_dict):
    cap_dem_dict = {}
    for i in range(num_airports):
        cap_dem_dict['req_{}'.format(i)] = cap_dict['req_{}'.format(i)] - pot_dem_dict['req_{}'.format(i)]
    return cap_dem_dict

[1. 0.]
[0. 1.]


In [10]:
class TwoAirportSchedEnv(gymnasium.Env):
    def __init__(self, number_of_actions, number_of_requests, num_airports, cap_per_airport_arr):
        super(TwoAirportSchedEnv, self).__init__()
        self.number_of_actions = int(number_of_actions)
        self.number_of_requests = number_of_requests
        self.num_airports = num_airports
        self.cap_per_airport_arr = cap_per_airport_arr
        self.cap_dict = generate_deterministic_capacity_dict(self.num_airports, self.cap_per_airport_arr)
        _exceed_cap = 0         
        while _exceed_cap == 0:
            self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = generate_scenario(number_of_requests = self.number_of_requests, num_airports = self.num_airports, cap_dict = self.cap_dict)
            self.cap_dem_dict_flat = flatten_cap_dem_dict(self.cap_dem_dict, self.num_airports)
            for i in range(num_airports):
                _exceed_cap = _exceed_cap + (min(self.cap_dem_dict_flat['req_{}'.format(i)]))

        # initialize the status capacity of all requests in requests_full
        self.initialize_status_capacity(self.cap_dem_dict, self.requests_full)
        # obtain the violate_set
        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)
        #to choose the request that will be checked
        self.get_req(self.requests_full, self.violate_set)

        #to generate action:
        self.action_space = spaces.Discrete(self.number_of_actions)
        self.observation_space = spaces.Box(low= -np.inf, high= np.inf, shape=(26,), dtype=float)
        self.num_step = 0
        
        self.original_requests = self.requests_full

    def initialize_status_capacity(self, cap_dem_dict, requests_full):
        for request_index in range(len(requests_full)):
            ts_dep = int(requests_full[request_index, 1])
            ts_arv = int(requests_full[request_index, 6])
            origin_airport = int(requests_full[request_index, 2].index(1.0))
            destination_airport = int(requests_full[request_index, 3].index(1.0))

            cap_dem_value_dep = cap_dem_dict['req_{}'.format(origin_airport)][ts_dep]
            cap_dem_value_arv = cap_dem_dict['req_{}'.format(destination_airport)][ts_arv]
            
            if cap_dem_value_dep >= 0:
                requests_full[request_index, 5] = 0
            if cap_dem_value_dep < 0:
                requests_full[request_index, 5] = 1
            if cap_dem_value_arv >= 0:
                requests_full[request_index, 7] = 0
            if cap_dem_value_arv < 0:
                requests_full[request_index, 7] = 1
    
    def get_violate_id_set(self, cap_dem_dict, requests_full):
        violate_set = []
        
        for request_index in range(len(requests_full)):
            status_cap_dep = requests_full[request_index, 5]
            status_cap_arv = requests_full[request_index, 7]

            if status_cap_dep == 1 or status_cap_arv == 1:
                violate_set.append(request_index)
        
        return violate_set
    
    #added get request to know which is the request we are going to move at this step",
    #Randomly choose a violating request to solve
    def get_req(self, requests_full, violate_set):
        if violate_set:
            chosen_request_id = random.choice(violate_set)
        else:
            print("The violate_set is empty. No value to select.")
            chosen_request_id = 0
        
        self.chosen_req = self.requests_full[self.requests_full[:,0] == chosen_request_id][0]

        return self.chosen_req
    
    def action_mapping(self, action, number_of_actions):
        mid_point = (number_of_actions - 1) // 2
        return action - mid_point
    
    #need to check arrival time or change the variable to check_outbound(arv) and chec_outbound(dep)
    def check_outbound(self, action):
        dep_time_slot = self.chosen_req[1]
        arv_time_slot = self.chosen_req[6]

        outbound = False

        new_dep_time_slot = dep_time_slot + action
        new_arv_time_slot = arv_time_slot + action

        if (new_dep_time_slot < 0) or (new_dep_time_slot > 287) or (new_arv_time_slot < 0) or (new_arv_time_slot > 287):
            outbound = True

        return outbound
    
    def dep_or_arv(self):
        pass

    def update_dem(self, dep_airport, arv_airport, time_slot_dep, time_slot_arv, new_time_slot_dep, new_time_slot_arv):
        # Increment demand for the new time slot for both departure and arrival airports
        self.pot_dem_dict['req_{}'.format(dep_airport)][new_time_slot_dep] += 1
        self.pot_dem_dict['req_{}'.format(arv_airport)][new_time_slot_arv] += 1

        # Decrease demand for the initial time slot for both departure and arrival airports
        self.pot_dem_dict['req_{}'.format(dep_airport)][time_slot_dep] -= 1
        self.pot_dem_dict['req_{}'.format(arv_airport)][time_slot_arv] -= 1

    def update_cap_dem(self):
        for airport in range(self.num_airports):
            cap_key = 'req_{}'.format(airport)
            dem_key = 'req_{}'.format(airport)
            if cap_key not in self.cap_dict:
                raise KeyError(f"'{cap_key}' not found in cap_dict. Available keys: {list(self.cap_dict.keys())}")
            self.cap_dem_dict[cap_key] = self.cap_dict[cap_key] - self.pot_dem_dict[dem_key]

    def get_airport_index(self, airport_type):
        # Check for airport type
        if airport_type == "departure":
            airport_index = self.chosen_req[2].index(1.0)
        elif airport_type == "arrival":
            airport_index = self.chosen_req[3].index(1.0)
        else:
            raise ValueError(f"Invalid airport_type: {airport_type}. Expected 'departure' or 'arrival'.")

        return airport_index
    
    def when_take_action(self, action):   
        _index = self.chosen_req[0]
        time_slot_dep = self.chosen_req[1]
        time_slot_arv = self.chosen_req[6]
        time_slot_arv = int(time_slot_arv)
        _dep_airport = self.chosen_req[2].index(1.0)
        _arv_airport = self.chosen_req[3].index(1.0)
        new_time_slot_dep = time_slot_dep + action
        new_time_slot_arv = time_slot_arv + action
        new_time_slot_arv = int(new_time_slot_arv)

        outbound = self.check_outbound(action)

        if not outbound:
            pass
            
        elif outbound:
            # new_time_slot_dep = random.randint(0, 287)
            # new_time_slot_arv = random.randint(0, 287)
            new_time_slot_dep = time_slot_dep
            new_time_slot_arv = time_slot_arv

        else:
            print('Problem with check outbound!')

        # Update the chosen request for both departure and arrival time slots
        self.chosen_req[1] = new_time_slot_dep
        self.chosen_req[6] = new_time_slot_arv

        # Update demand for the new time slots
        self.update_dem(_dep_airport, _arv_airport, time_slot_dep, time_slot_arv, new_time_slot_dep, new_time_slot_arv)

        # Update cap_dem
        self.update_cap_dem()
        
        if self.cap_dem_dict['req_{}'.format(_dep_airport)][new_time_slot_dep] >= 0:
            self.chosen_req[5] = 0
        else:
            self.chosen_req[5] = 1

        if self.cap_dem_dict['req_{}'.format(_arv_airport)][new_time_slot_arv] >= 0:
            self.chosen_req[7] = 0
        else:
            self.chosen_req[7] = 1
        
        self.requests_full[self.requests_full[:, 0] == _index] = self.chosen_req

        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)

    def step(self, action): 
        num_actions = self.number_of_actions
        # # Convert the discrete action to your desired action value
        real_action = self.action_mapping(action, num_actions)

        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        
        # observation concats the following:
        # ts_dep + cap_dem_dep + ts_arv + cap_dem_arv
        self.when_take_action(real_action)
        outbound = self.check_outbound(real_action)

        # Count number of remaining unaccommodated requests
        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        
        # Reward part:
        done = False
        self.num_step += 1
        
        if outbound:
            real_action = 0
            
        if real_action != 0:
            local_reward = 0.1*(-abs(real_action))

            if (_unaccom_req == 0 or len(self.violate_set) == 0) and self.num_step != self.number_of_requests*5:
                done = True
                self.num_step = 0
                obs = np.zeros(26)
                global_reward = 30

            elif self.num_step == self.number_of_requests*5:
                done = True
                self.num_step = 0
                obs = np.zeros(26)
                global_reward = _unaccom_req * (-1)

            else:
                global_reward = 0
                obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)
            reward_time_step = -0.8
        
        else:
            reward_time_step = -0.8
            local_reward = 0
            global_reward = 0
            obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)

        total_reward = float(local_reward + global_reward + reward_time_step)

        truncated = False
    
        return obs, total_reward, done, truncated, {}

    def _next_observation(self, airport_req_dict, num_airports, cap_dem_dict, requests_full):

        # Check if violate_set is empty before getting a request
        if len(self.violate_set) == 0:
            # self.chosen_req = None
            # Return a default observation or handle as necessary
            return np.zeros(26)
        
        # To choose the request that will be checked
        self.get_req(self.requests_full, self.violate_set)
        _ts_dep = int(self.chosen_req[1])  # Ensure _ts_dep is an integer
        _ts_arv = int(self.chosen_req[6])  # Ensure _ts_arv is an integer
        _dep_airport = self.chosen_req[2].index(1.0)
        _arv_airport = self.chosen_req[3].index(1.0)
        cap_dem_arr_dep = self.cap_dem_dict['req_{}'.format(_dep_airport)]
        cap_dem_arr_arv = self.cap_dem_dict['req_{}'.format(_arv_airport)]
    
        # Function to safely get the time slot range with edge case handling
        def get_time_slots(array, time_slot):
            # Initialize an array of -100s
            time_slots = np.full(13, -100.0)
    
            start = max(0, time_slot - 6)
            end = min(len(array), time_slot + 7)  # +7 because the end index is exclusive
            actual_slots = array[start:end]
    
            # Calculate the offset for start and end
            start_offset = 6 - (time_slot - start)
            end_offset = 6 + (end - time_slot)
    
            # Fill the actual values from the array into the initialized array
            time_slots[start_offset:start_offset + len(actual_slots)] = actual_slots
    
            return time_slots
    
        # Getting the relevant time slots for departure and arrival
        relevant_dep = get_time_slots(cap_dem_arr_dep, _ts_dep)
        relevant_arv = get_time_slots(cap_dem_arr_arv, _ts_arv)
    
        # Explicitly declare self.obs as a fixed-size array of length 26
        self.obs = np.empty(26)
        self.obs[:13] = relevant_dep
        self.obs[13:] = relevant_arv
        
        return self.obs

    def reset(self, seed=None):                
        self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = generate_scenario(self.number_of_requests, self.num_airports, cap_dict = self.cap_dict)
        self.original_requests = self.requests_full.copy()
        cap_dem_arr_airport1 = self.cap_dem_dict['req_{}'.format(0)]
        cap_dem_arr_airport2 = self.cap_dem_dict['req_{}'.format(1)]
        
        _cap_dem_flat_airport1 = cap_dem_arr_airport1.flatten()
        _cap_dem_flat_airport2 = cap_dem_arr_airport2.flatten()

        while (min(_cap_dem_flat_airport1) >= 0 or min(_cap_dem_flat_airport2) >= 0):
            self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = generate_scenario(self.number_of_requests, self.num_airports, cap_dict = self.cap_dict)
            _cap_dem_flat_airport1 = cap_dem_arr_airport1.flatten()
            _cap_dem_flat_airport2 = cap_dem_arr_airport2.flatten()
        
        # initialize the status capacity of all requests in requests_full
        self.initialize_status_capacity(self.cap_dem_dict, self.requests_full)
        # obtain the violate_set
        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)
        
        # print('Number of violation for airport1: ', len(_cap_dem_flat_airport1[_cap_dem_flat_airport1 < 0]))
        # print('Number of violation for airport2: ', len(_cap_dem_flat_airport2[_cap_dem_flat_airport2 < 0]))

        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        print("Number of unaccomodated requests at the start of current episode:", _unaccom_req)

        self.num_step = 0
        info = {}
        return self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full), info
    
    # Need to validate
    def evaluate_metrics(self):
        _initial_cap_dem_dict = {}
        _initial_pot_dem_dict = get_initial_pot_dem_per_airport(self.airport_req_dict, self.num_airports)
        for airport in range(self.num_airports):
            cap_key = 'req_{}'.format(airport)
            dem_key = 'req_{}'.format(airport)
            _initial_cap_dem_dict[cap_key] = self.cap_dict[cap_key] - _initial_pot_dem_dict[dem_key]

        # Code to print initial number of violation requests for each airport agent
        _initial_cap_dem_airport1 = _initial_cap_dem_dict['req_{}'.format(0)]
        _initial_cap_dem_flat_airport1 = _initial_cap_dem_airport1.flatten()
        _initial_violate_airport1 = len(_initial_cap_dem_flat_airport1[_initial_cap_dem_flat_airport1 < 0])
        print('Initial count of unit violation for airport1 is: ', _initial_violate_airport1)
        _initial_cap_dem_airport2 = _initial_cap_dem_dict['req_{}'.format(1)]
        _initial_cap_dem_flat_airport2 = _initial_cap_dem_airport2.flatten()
        _initial_violate_airport2 = len(_initial_cap_dem_flat_airport2[_initial_cap_dem_flat_airport2 < 0])
        print('Initial count of unit violation for airport2 is: ', _initial_violate_airport2)

        # Code to print final number of violation requests for each airport agent
        cap_dem_arr_airport1 = self.cap_dem_dict['req_{}'.format(0)]
        cap_dem_arr_airport2 = self.cap_dem_dict['req_{}'.format(1)]
        _final_cap_dem_flat_airport1 = cap_dem_arr_airport1.flatten()
        _final_cap_dem_flat_airport2 = cap_dem_arr_airport2.flatten()
        _final_violate_airport1 = len(_final_cap_dem_flat_airport1[_final_cap_dem_flat_airport1 < 0])
        _final_violate_airport2 = len(_final_cap_dem_flat_airport2[_final_cap_dem_flat_airport2 < 0])
        print('Final count of unit violation for airport1 is: ', _final_violate_airport1)
        print('Final count of unit violation for airport2 is: ', _final_violate_airport2)

        _total_sched_delay = sum(abs(self.original_requests[:,1] - self.requests_full[:,1]))  
        print('Total schedule delay is: ', _total_sched_delay)

        _max_shift = max(abs(self.original_requests[:,1] - self.requests_full[:,1]))
        print('Max shift: ', _max_shift)

        # Code to print number of unaccommodated requests at the end of the episode
        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        print('Number of unaccommodated requests: ', _unaccom_req)

        return _initial_violate_airport1, _initial_violate_airport2, _final_violate_airport1, _final_violate_airport2, _total_sched_delay, _max_shift, _unaccom_req

    def set_scenario(self, scenario):
        # Extract components from the scenario tuple
        self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = scenario
        self.original_requests = self.requests_full.copy()

        # Initialize the status capacity of all requests in requests_full
        self.initialize_status_capacity(self.cap_dem_dict, self.requests_full)

        # Obtain the violate_set
        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)

        # Reset step count
        self.num_step = 0

        # Generate the initial observation
        obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)

        self._unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])

        # Return the initial observation and additional info (if any)
        return obs, {}

# Setting 4

In [3]:
#Function for normal distribution truncation:

from scipy.stats import truncnorm

def get_truncated_normal(mean, sd, low, upp):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

#Function to get observation (still for single env, need to modify for multi-agent env):

def full_obs(_cap_dem_chosen_req, number_of_actions):
    _obs_min_arr = np.full((288+number_of_actions-1, ), 0)
    for i in range(len(_cap_dem_chosen_req)):
        _obs_min_arr[i+int((number_of_actions-1)/2)] = min(_cap_dem_chosen_req[i])
    return _obs_min_arr

#Function to get the one-hot-encoded vectors for departure and arrival airports:

def one_hot_encode_airport(airport, num_airports):
    encoding = np.zeros(num_airports)
    encoding[airport] = 1
    return encoding

# Example usage
num_airports = 2
airport1 = 0
airport2 = 1

encoded_airport1 = one_hot_encode_airport(airport1, num_airports)
encoded_airport2 = one_hot_encode_airport(airport2, num_airports)

print(encoded_airport1)
print(encoded_airport2)

#Generate full info for the arrival sides:

def generate_info_arv(requests):
    ts_arv = np.empty(shape=(len(requests),), dtype='object')
    for i in range(len(requests)):
        ts_arv[i] = requests[i][1] + requests[i][4]/5
        if ts_arv[i] > 287:
            ts_arv[i] = ts_arv[i] - 287
    return ts_arv

#Generate the multi-agent scenario:

#=======================================================

#Modify the distribution based on historical data later:
def generate_scenario(number_of_requests, num_airports, cap_dict):
    
    #number_of_requests = 1500
    ts_72 = get_truncated_normal(mean=72, sd=36, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_72 = np.round(ts_72)

    ts_216 = get_truncated_normal(mean=216, sd=36, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_216 = np.round(ts_216)

    ts_dep = np.concatenate((ts_72, ts_216))
    ts_dep = ts_dep.astype(int)

    #Generate index for requests:

    index = np.array(list(range(number_of_requests)))

    #Generate origin (0 and 1 are two considered origin airports, 2 represent other airports, encoded in one-hot vector):

    #num_airports = 3
    origin_airport = np.empty(shape=(number_of_requests,), dtype='object')
    destination_airport = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range(number_of_requests):
        _org_airport = one_hot_encode_airport(random.randint(0,1), num_airports)
        _org_airport_list = _org_airport.tolist()
        origin_airport[i] = _org_airport_list
        #Generate destination (the destination will be different with the origin):
        _dest_airport = _org_airport.copy()
        while np.array_equal(_dest_airport, _org_airport):
            np.random.shuffle(_dest_airport)
        _dest_airport_list = _dest_airport.tolist()
        destination_airport[i] = _dest_airport_list

    #Generate flying time (assume between airport 0 and 1 is 2 hour, 0 to 2 and 1 to 2 is arbitrary):

    fly_time = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range (number_of_requests):
        if origin_airport[i] == list([1.0, 0.0]) and destination_airport[i] == list([0.0, 1.0]):
            fly_time[i] = 120
        elif origin_airport[i] == list([0.0, 1.0]) and destination_airport[i] == list([1.0, 0.0]):
            fly_time[i] = random.choice([60, 120, 180])

    #Generate status cap:

    status_cap_dep = np.full((number_of_requests,), 0)
    status_cap_arv = np.full((number_of_requests,), 0)
    

    requests = np.stack((index, ts_dep, origin_airport, destination_airport, fly_time, status_cap_dep), axis=1)

    #Generate full info for the arv side:

    ts_arv = generate_info_arv(requests)

    #pseudo_belong_dep = np.full((number_of_requests,), 0)
    #pseudo_belong_arv = np.full((number_of_requests,), 0)
    
    # Define requests_full as dtype object
    num_entries = len(index)  # Given that 'index' is defined using np.array(list(range(number_of_requests)))
    # Create an empty array of the desired shape with dtype=object
    requests_full = np.empty((num_entries, 8), dtype=object)
    # Fill the array
    data = [index, ts_dep, origin_airport, destination_airport, fly_time, status_cap_dep, ts_arv, status_cap_arv]
    for i, column_data in enumerate(data):
        requests_full[:, i] = column_data

    # airport_req_dict: A dictionary where each key corresponds to a specific airport's requirements. 
    # The key format is 'req_i', where i is the index of the airport. 
    # The value for each key is a numpy array, with each row representing a request and the columns containing different attributes of that request.
    airport_req_dict, _belong_airport_dict = get_airport_req_dict(requests_full, num_airports)

    pot_dem_dict = get_initial_pot_dem_per_airport(airport_req_dict, num_airports)

    cap_dem_dict = get_cap_dem_dict(num_airports, cap_dict, pot_dem_dict)

    return requests_full, airport_req_dict, _belong_airport_dict, pot_dem_dict, cap_dem_dict

def flatten_cap_dem_dict(cap_dem_dict, num_airports):
    cap_dem_dict_flat = {}
    for i in range(num_airports):
        cap_dem_dict_flat['req_{}'.format(i)] = cap_dem_dict['req_{}'.format(i)].flatten()
    return cap_dem_dict_flat

#Get separated req per airport and store in a dict:

def get_airport_req_dict(requests_full, num_airports):
    airport_req_dict = {}
    _belong_airport_dict = {}
    for i in range(num_airports):
        airport_req_dict['req_{}'.format(i)] = np.empty((0, 10)) #This one depends on the number of elements of a final request
        _belong_airport_dict['req_{}'.format(i)] = np.full(num_airports, 0.0, dtype=float)
        _belong_airport_dict['req_{}'.format(i)][i] = float(1.0)
        _belong_airport_dict['req_{}'.format(i)] = _belong_airport_dict['req_{}'.format(i)].tolist()
        
    for i in range(len(requests_full)):
        _found_dep = 0
        _found_arv = 0
        for k in range(num_airports):
            #_found_dep = 0
            #_found_arv = 0
            if requests_full[i][2] == _belong_airport_dict['req_{}'.format(k)]:
                _dep_req = np.append(requests_full[i], 1)
                _dep_req = np.append(_dep_req, 0)
                airport_req_dict['req_{}'.format(k)] = np.vstack((airport_req_dict['req_{}'.format(k)], _dep_req))
                _found_dep = 1

            if requests_full[i][3] == _belong_airport_dict['req_{}'.format(k)]:
                _arv_req = np.append(requests_full[i], 0)
                _arv_req = np.append(_arv_req, 1)
                airport_req_dict['req_{}'.format(k)] = np.vstack((airport_req_dict['req_{}'.format(k)], _arv_req))
                _found_arv = 1

            if _found_dep + _found_arv == 2:
                break
        if _found_dep + _found_arv != 2:
            print("Cannot found both dep and arv at req {}".format(i))
    
    return airport_req_dict, _belong_airport_dict

def generate_deterministic_capacity_dict(num_airports, cap_per_airport_arr): #This function is for 288 slots/ day
    cap_dict = {}
    for i in range(num_airports):
        cap_dict['req_{}'.format(i)] = np.full((288), cap_per_airport_arr[i])
    return cap_dict

def get_date_plot(timeslot_movements, i):
    indices, items = zip(*timeslot_movements)
    sum_of_values = sum(item[1] for item in timeslot_movements)
    print("no of request: ", sum_of_values)
    plt.figure(figsize=(10,6))
    plt.bar(indices, items, width=3.0) 
 
    plt.xlabel('Timeslot') 
    plt.ylabel('No. of movements') 

    plt.title('No. of movements by timeslot in airport' +  str(i)) 

    plt.show()

def get_initial_pot_dem_per_airport(airport_req_dict, num_airports): #Replace req_df to req_df_update to update pot_dem_df #To be replaced with final_sched
    pot_dem_dict = {}
    #TODO: increase speed
    #8 dep 9 arv, 1 dep ts, 6 arv ts
    for i in range(num_airports):
        pot_dem_dict['req_{}'.format(i)] = np.full((288), 0)
        for k in range(len(airport_req_dict['req_{}'.format(i)])):
            _time_slot = int(airport_req_dict['req_{}'.format(i)][k][1]) * int(airport_req_dict['req_{}'.format(i)][k][8]) + int(airport_req_dict['req_{}'.format(i)][k][6]) * int(airport_req_dict['req_{}'.format(i)][k][9])
            pot_dem_dict['req_{}'.format(i)][_time_slot] += 1

    # for i in range(num_airports):
    #     indexed_array = [[index, item] for index, item in enumerate(pot_dem_dict['req_{}'.format(i)])]
    #     get_date_plot(indexed_array, i)
        
    return pot_dem_dict

def get_cap_dem_dict(num_airports, cap_dict, pot_dem_dict):
    cap_dem_dict = {}
    for i in range(num_airports):
        cap_dem_dict['req_{}'.format(i)] = cap_dict['req_{}'.format(i)] - pot_dem_dict['req_{}'.format(i)]
    return cap_dem_dict

[1. 0.]
[0. 1.]


In [15]:
class TwoAirportSchedEnv(gymnasium.Env):
    def __init__(self, number_of_actions, number_of_requests, num_airports, cap_per_airport_arr):
        super(TwoAirportSchedEnv, self).__init__()
        self.number_of_actions = int(number_of_actions)
        self.number_of_requests = number_of_requests
        self.num_airports = num_airports
        self.cap_per_airport_arr = cap_per_airport_arr
        self.cap_dict = generate_deterministic_capacity_dict(self.num_airports, self.cap_per_airport_arr)
        _exceed_cap = 0         
        while _exceed_cap == 0:
            self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = generate_scenario(number_of_requests = self.number_of_requests, num_airports = self.num_airports, cap_dict = self.cap_dict)
            self.cap_dem_dict_flat = flatten_cap_dem_dict(self.cap_dem_dict, self.num_airports)
            for i in range(num_airports):
                _exceed_cap = _exceed_cap + (min(self.cap_dem_dict_flat['req_{}'.format(i)]))

        # initialize the status capacity of all requests in requests_full
        self.initialize_status_capacity(self.cap_dem_dict, self.requests_full)
        # obtain the violate_set
        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)
        #to choose the request that will be checked
        self.get_req(self.requests_full, self.violate_set)

        #to generate action:
        self.action_space = spaces.Discrete(self.number_of_actions)
        self.observation_space = spaces.Box(low= -np.inf, high= np.inf, shape=(578,), dtype=float)
        self.num_step = 0
        
        self.original_requests = self.requests_full

    def initialize_status_capacity(self, cap_dem_dict, requests_full):
        for request_index in range(len(requests_full)):
            ts_dep = int(requests_full[request_index, 1])
            ts_arv = int(requests_full[request_index, 6])
            origin_airport = int(requests_full[request_index, 2].index(1.0))
            destination_airport = int(requests_full[request_index, 3].index(1.0))

            cap_dem_value_dep = cap_dem_dict['req_{}'.format(origin_airport)][ts_dep]
            cap_dem_value_arv = cap_dem_dict['req_{}'.format(destination_airport)][ts_arv]
            
            if cap_dem_value_dep >= 0:
                requests_full[request_index, 5] = 0
            if cap_dem_value_dep < 0:
                requests_full[request_index, 5] = 1
            if cap_dem_value_arv >= 0:
                requests_full[request_index, 7] = 0
            if cap_dem_value_arv < 0:
                requests_full[request_index, 7] = 1
    
    def get_violate_id_set(self, cap_dem_dict, requests_full):
        violate_set = []
        
        for request_index in range(len(requests_full)):
            status_cap_dep = requests_full[request_index, 5]
            status_cap_arv = requests_full[request_index, 7]

            if status_cap_dep == 1 or status_cap_arv == 1:
                violate_set.append(request_index)
        
        return violate_set
    
    #added get request to know which is the request we are going to move at this step",
    #Randomly choose a violating request to solve
    def get_req(self, requests_full, violate_set):
        if violate_set:
            chosen_request_id = random.choice(violate_set)
        else:
            print("The violate_set is empty. No value to select.")
            chosen_request_id = 0
        
        self.chosen_req = self.requests_full[self.requests_full[:,0] == chosen_request_id][0]

        return self.chosen_req
    
    def action_mapping(self, action, number_of_actions):
        mid_point = (number_of_actions - 1) // 2
        return action - mid_point
    
    #need to check arrival time or change the variable to check_outbound(arv) and chec_outbound(dep)
    def check_outbound(self, action):
        dep_time_slot = self.chosen_req[1]
        arv_time_slot = self.chosen_req[6]

        outbound = False

        new_dep_time_slot = dep_time_slot + action
        new_arv_time_slot = arv_time_slot + action

        if (new_dep_time_slot < 0) or (new_dep_time_slot > 287) or (new_arv_time_slot < 0) or (new_arv_time_slot > 287):
            outbound = True

        return outbound
    
    def dep_or_arv(self):
        pass

    def update_dem(self, dep_airport, arv_airport, time_slot_dep, time_slot_arv, new_time_slot_dep, new_time_slot_arv):
        # Increment demand for the new time slot for both departure and arrival airports
        self.pot_dem_dict['req_{}'.format(dep_airport)][new_time_slot_dep] += 1
        self.pot_dem_dict['req_{}'.format(arv_airport)][new_time_slot_arv] += 1

        # Decrease demand for the initial time slot for both departure and arrival airports
        self.pot_dem_dict['req_{}'.format(dep_airport)][time_slot_dep] -= 1
        self.pot_dem_dict['req_{}'.format(arv_airport)][time_slot_arv] -= 1

    def update_cap_dem(self):
        for airport in range(self.num_airports):
            cap_key = 'req_{}'.format(airport)
            dem_key = 'req_{}'.format(airport)
            if cap_key not in self.cap_dict:
                raise KeyError(f"'{cap_key}' not found in cap_dict. Available keys: {list(self.cap_dict.keys())}")
            self.cap_dem_dict[cap_key] = self.cap_dict[cap_key] - self.pot_dem_dict[dem_key]

    def get_airport_index(self, airport_type):
        # Check for airport type
        if airport_type == "departure":
            airport_index = self.chosen_req[2].index(1.0)
        elif airport_type == "arrival":
            airport_index = self.chosen_req[3].index(1.0)
        else:
            raise ValueError(f"Invalid airport_type: {airport_type}. Expected 'departure' or 'arrival'.")

        return airport_index
    
    def when_take_action(self, action):   
        _index = self.chosen_req[0]
        time_slot_dep = self.chosen_req[1]
        time_slot_arv = self.chosen_req[6]
        time_slot_arv = int(time_slot_arv)
        _dep_airport = self.chosen_req[2].index(1.0)
        _arv_airport = self.chosen_req[3].index(1.0)
        new_time_slot_dep = time_slot_dep + action
        new_time_slot_arv = time_slot_arv + action
        new_time_slot_arv = int(new_time_slot_arv)

        outbound = self.check_outbound(action)

        if not outbound:
            pass
            
        elif outbound:
            # new_time_slot_dep = random.randint(0, 287)
            # new_time_slot_arv = random.randint(0, 287)
            new_time_slot_dep = time_slot_dep
            new_time_slot_arv = time_slot_arv

        else:
            print('Problem with check outbound!')

        # Update the chosen request for both departure and arrival time slots
        self.chosen_req[1] = new_time_slot_dep
        self.chosen_req[6] = new_time_slot_arv

        # Update demand for the new time slots
        self.update_dem(_dep_airport, _arv_airport, time_slot_dep, time_slot_arv, new_time_slot_dep, new_time_slot_arv)

        # Update cap_dem
        self.update_cap_dem()
        
        if self.cap_dem_dict['req_{}'.format(_dep_airport)][new_time_slot_dep] >= 0:
            self.chosen_req[5] = 0
        else:
            self.chosen_req[5] = 1

        if self.cap_dem_dict['req_{}'.format(_arv_airport)][new_time_slot_arv] >= 0:
            self.chosen_req[7] = 0
        else:
            self.chosen_req[7] = 1
        
        self.requests_full[self.requests_full[:, 0] == _index] = self.chosen_req

        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)

    def step(self, action): 
        num_actions = self.number_of_actions
        # # Convert the discrete action to your desired action value
        real_action = self.action_mapping(action, num_actions)

        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        
        # observation concats the following:
        # ts_dep + cap_dem_dep + ts_arv + cap_dem_arv
        self.when_take_action(real_action)
        outbound = self.check_outbound(real_action)

        # Count number of remaining unaccommodated requests
        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        
        # Reward part:
        done = False
        self.num_step += 1
        
        if outbound:
            real_action = 0
            
        if real_action != 0:
            local_reward = 0.1*(-abs(real_action))
            
            if self.chosen_req[5] == 0 and self.chosen_req[7] == 0:
                reward_solved = 1

            else:
                reward_solved = 0

            if (_unaccom_req == 0 or len(self.violate_set) == 0) and self.num_step != self.number_of_requests*5:
                done = True
                self.num_step = 0
                obs = np.zeros(578)
                global_reward = 30

            elif self.num_step == self.number_of_requests*5:
                done = True
                self.num_step = 0
                obs = np.zeros(578)
                global_reward = _unaccom_req * (-1)

            else:
                global_reward = 0
                obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)
            reward_time_step = -0.8
        
        else:
            reward_time_step = -0.8
            local_reward = 0
            global_reward = 0
            reward_solved = 0
            obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)

        total_reward = float(local_reward + reward_solved + global_reward + reward_time_step)

        truncated = False
    
        return obs, total_reward, done, truncated, {}

    # def _next_observation(self, airport_req_dict, num_airports, cap_dem_dict, requests_full):

    #     # Check if violate_set is empty before getting a request
    #     if len(self.violate_set) == 0:
    #         # self.chosen_req = None
    #         # Return a default observation or handle as necessary
    #         return np.zeros(26)
        
    #     # To choose the request that will be checked
    #     self.get_req(self.requests_full, self.violate_set)
    #     _ts_dep = int(self.chosen_req[1])  # Ensure _ts_dep is an integer
    #     _ts_arv = int(self.chosen_req[6])  # Ensure _ts_arv is an integer
    #     _dep_airport = self.chosen_req[2].index(1.0)
    #     _arv_airport = self.chosen_req[3].index(1.0)
    #     cap_dem_arr_dep = self.cap_dem_dict['req_{}'.format(_dep_airport)]
    #     cap_dem_arr_arv = self.cap_dem_dict['req_{}'.format(_arv_airport)]
    
    #     # Function to safely get the time slot range with edge case handling
    #     def get_time_slots(array, time_slot):
    #         # Initialize an array of -100s
    #         time_slots = np.full(13, -100.0)
    
    #         start = max(0, time_slot - 6)
    #         end = min(len(array), time_slot + 7)  # +7 because the end index is exclusive
    #         actual_slots = array[start:end]
    
    #         # Calculate the offset for start and end
    #         start_offset = 6 - (time_slot - start)
    #         end_offset = 6 + (end - time_slot)
    
    #         # Fill the actual values from the array into the initialized array
    #         time_slots[start_offset:start_offset + len(actual_slots)] = actual_slots
    
    #         return time_slots
    
    #     # Getting the relevant time slots for departure and arrival
    #     relevant_dep = get_time_slots(cap_dem_arr_dep, _ts_dep)
    #     relevant_arv = get_time_slots(cap_dem_arr_arv, _ts_arv)
    
    #     # Explicitly declare self.obs as a fixed-size array of length 26
    #     self.obs = np.empty(26)
    #     self.obs[:13] = relevant_dep
    #     self.obs[13:] = relevant_arv
        
    #     return self.obs

    def _next_observation(self, airport_req_dict, num_airports, cap_dem_dict, requests_full):
        #to choose the request that will be checked
        self.get_req(self.requests_full, self.violate_set)
        _ts_dep = self.chosen_req[1]
        _ts_arv = self.chosen_req[6]
        _dep_airport = self.chosen_req[2].index(1.0)
        _arv_airport = self.chosen_req[3].index(1.0)
        cap_dem_arr_dep = self.cap_dem_dict['req_{}'.format(_dep_airport)]
        cap_dem_arr_arv = self.cap_dem_dict['req_{}'.format(_arv_airport)]

        self.obs = np.append(_ts_dep, cap_dem_arr_dep)
        self.obs = np.append(self.obs, _ts_arv)
        self.obs = np.append(self.obs, cap_dem_arr_arv)
        
        return self.obs

    def reset(self, seed=None):                
        self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = generate_scenario(self.number_of_requests, self.num_airports, cap_dict = self.cap_dict)
        self.original_requests = self.requests_full.copy()
        cap_dem_arr_airport1 = self.cap_dem_dict['req_{}'.format(0)]
        cap_dem_arr_airport2 = self.cap_dem_dict['req_{}'.format(1)]
        
        _cap_dem_flat_airport1 = cap_dem_arr_airport1.flatten()
        _cap_dem_flat_airport2 = cap_dem_arr_airport2.flatten()

        while (min(_cap_dem_flat_airport1) >= 0 or min(_cap_dem_flat_airport2) >= 0):
            self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = generate_scenario(self.number_of_requests, self.num_airports, cap_dict = self.cap_dict)
            _cap_dem_flat_airport1 = cap_dem_arr_airport1.flatten()
            _cap_dem_flat_airport2 = cap_dem_arr_airport2.flatten()
        
        # initialize the status capacity of all requests in requests_full
        self.initialize_status_capacity(self.cap_dem_dict, self.requests_full)
        # obtain the violate_set
        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)
        
        # print('Number of violation for airport1: ', len(_cap_dem_flat_airport1[_cap_dem_flat_airport1 < 0]))
        # print('Number of violation for airport2: ', len(_cap_dem_flat_airport2[_cap_dem_flat_airport2 < 0]))

        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        print("Number of unaccomodated requests at the start of current episode:", _unaccom_req)

        self.num_step = 0
        info = {}
        return self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full), info
    
    # Need to validate
    def evaluate_metrics(self):
        _initial_cap_dem_dict = {}
        _initial_pot_dem_dict = get_initial_pot_dem_per_airport(self.airport_req_dict, self.num_airports)
        for airport in range(self.num_airports):
            cap_key = 'req_{}'.format(airport)
            dem_key = 'req_{}'.format(airport)
            _initial_cap_dem_dict[cap_key] = self.cap_dict[cap_key] - _initial_pot_dem_dict[dem_key]

        # Code to print initial number of violation requests for each airport agent
        _initial_cap_dem_airport1 = _initial_cap_dem_dict['req_{}'.format(0)]
        _initial_cap_dem_flat_airport1 = _initial_cap_dem_airport1.flatten()
        _initial_violate_airport1 = len(_initial_cap_dem_flat_airport1[_initial_cap_dem_flat_airport1 < 0])
        print('Initial count of unit violation for airport1 is: ', _initial_violate_airport1)
        _initial_cap_dem_airport2 = _initial_cap_dem_dict['req_{}'.format(1)]
        _initial_cap_dem_flat_airport2 = _initial_cap_dem_airport2.flatten()
        _initial_violate_airport2 = len(_initial_cap_dem_flat_airport2[_initial_cap_dem_flat_airport2 < 0])
        print('Initial count of unit violation for airport2 is: ', _initial_violate_airport2)

        # Code to print final number of violation requests for each airport agent
        cap_dem_arr_airport1 = self.cap_dem_dict['req_{}'.format(0)]
        cap_dem_arr_airport2 = self.cap_dem_dict['req_{}'.format(1)]
        _final_cap_dem_flat_airport1 = cap_dem_arr_airport1.flatten()
        _final_cap_dem_flat_airport2 = cap_dem_arr_airport2.flatten()
        _final_violate_airport1 = len(_final_cap_dem_flat_airport1[_final_cap_dem_flat_airport1 < 0])
        _final_violate_airport2 = len(_final_cap_dem_flat_airport2[_final_cap_dem_flat_airport2 < 0])
        print('Final count of unit violation for airport1 is: ', _final_violate_airport1)
        print('Final count of unit violation for airport2 is: ', _final_violate_airport2)

        _total_sched_delay = sum(abs(self.original_requests[:,1] - self.requests_full[:,1]))  
        print('Total schedule delay is: ', _total_sched_delay)

        _max_shift = max(abs(self.original_requests[:,1] - self.requests_full[:,1]))
        print('Max shift: ', _max_shift)

        # Code to print number of unaccommodated requests at the end of the episode
        _unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])
        print('Number of unaccommodated requests: ', _unaccom_req)

        return _initial_violate_airport1, _initial_violate_airport2, _final_violate_airport1, _final_violate_airport2, _total_sched_delay, _max_shift, _unaccom_req

    def set_scenario(self, scenario):
        # Extract components from the scenario tuple
        self.requests_full, self.airport_req_dict, self._belong_airport_dict, self.pot_dem_dict, self.cap_dem_dict = scenario
        self.original_requests = self.requests_full.copy()

        # Initialize the status capacity of all requests in requests_full
        self.initialize_status_capacity(self.cap_dem_dict, self.requests_full)

        # Obtain the violate_set
        self.violate_set = self.get_violate_id_set(self.cap_dem_dict, self.requests_full)

        # Reset step count
        self.num_step = 0

        # Generate the initial observation
        obs = self._next_observation(self.airport_req_dict, self.num_airports, self.cap_dem_dict, self.requests_full)

        self._unaccom_req = len(self.requests_full[(self.requests_full[:, 5] == 1) | (self.requests_full[:, 7] == 1)])

        # Return the initial observation and additional info (if any)
        return obs, {}

In [16]:
# # Generate 20 scenarios
import random
import numpy as np

num_scenarios = 20
num_airports = 2
number_of_requests = 1000
cap_per_airport_arr = [6, 6]

# Set a fixed seed for reproducibility
random.seed(42)
np.random.seed(42)

# Generate scenarios
scenarios = [generate_scenario(number_of_requests, num_airports, generate_deterministic_capacity_dict(num_airports, cap_per_airport_arr)) for _ in range(num_scenarios)]

# StableBaselines Training Code

In [6]:
from stable_baselines3.common.env_checker import check_env
env = TwoAirportSchedEnv(number_of_actions=num_actions, number_of_requests=1100, num_airports=2, cap_per_airport_arr= [6,6])
for i in range(50):
    check_env(env)
    i += 1

Number of unaccomodated requests at the start of current episode: 548
Number of unaccomodated requests at the start of current episode: 523
Number of unaccomodated requests at the start of current episode: 583
Number of unaccomodated requests at the start of current episode: 557
Number of unaccomodated requests at the start of current episode: 487
Number of unaccomodated requests at the start of current episode: 490
Number of unaccomodated requests at the start of current episode: 527
Number of unaccomodated requests at the start of current episode: 525
Number of unaccomodated requests at the start of current episode: 522
Number of unaccomodated requests at the start of current episode: 516
Number of unaccomodated requests at the start of current episode: 553
Number of unaccomodated requests at the start of current episode: 496
Number of unaccomodated requests at the start of current episode: 497
Number of unaccomodated requests at the start of current episode: 529
Number of unaccomoda

In [7]:
from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import BaseCallback

#model = SAC("MlpPolicy", "Pendulum-v1", tensorboard_log="/tmp/sac/", verbose=1)


class TensorboardCallback(BaseCallback):
    """
    Custom callback for plotting additional values in tensorboard.
    """

    def __init__(self, verbose=0):
        super(TensorboardCallback, self).__init__(verbose)

    def _on_step(self) -> bool:
        # Log scalar value (here a random variable)
        value = np.random.random()
        self.logger.record('random_value', value)
        return True

In [8]:
env = TwoAirportSchedEnv(number_of_actions=num_actions, number_of_requests=1100, num_airports=2, cap_per_airport_arr=[6,6])
model_full_3 = DQN("MlpPolicy", env, tensorboard_log="./log/", verbose=1, exploration_fraction = 0.6)
model_full_3.learn(6000000, tb_log_name="DQN")
model_full_3.save("dqn_schedenv_full_3")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Number of unaccomodated requests at the start of current episode: 483
Logging to ./log/DQN_1
Number of unaccomodated requests at the start of current episode: 510
Number of unaccomodated requests at the start of current episode: 496
Number of unaccomodated requests at the start of current episode: 486
Number of unaccomodated requests at the start of current episode: 499
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 1.91e+03  |
|    ep_rew_mean      | -2.12e+03 |
|    exploration_rate | 0.998     |
| time/               |           |
|    episodes         | 4         |
|    fps              | 1627      |
|    time_elapsed     | 4         |
|    total_timesteps  | 7656      |
-----------------------------------
Number of unaccomodated requests at the start of current episode: 468
Number of unaccomodated requests at the start of current episode: 

# StableBaselines Testing Code

In [6]:
env = TwoAirportSchedEnv(number_of_actions=13, number_of_requests=1000, num_airports=2, cap_per_airport_arr=[6,6])
model = DQN.load("dqn_schedenv_full_setting1")

def evaluate_agent(scenarios):
    for scenario in scenarios:
        # Set the environment to the current scenario
        obs, info = env.set_scenario(scenario)
        counter = 0
        print('Initial Number of unaccommodated requests: ', env._unaccom_req)

        while True:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)

            if done or counter == 5000:
                env.evaluate_metrics()
                print("\n---------------------------------------------------------\n")
                break

            counter += 1

evaluate_agent(scenarios)

Initial Number of unaccommodated requests:  370
Initial count of unit violation for airport1 is:  31
Initial count of unit violation for airport2 is:  26
Final count of unit violation for airport1 is:  0
Final count of unit violation for airport2 is:  0
Total schedule delay is:  786
Max shift:  8
Number of unaccommodated requests:  0

---------------------------------------------------------

Initial Number of unaccommodated requests:  313
Initial count of unit violation for airport1 is:  25
Initial count of unit violation for airport2 is:  22
Final count of unit violation for airport1 is:  0
Final count of unit violation for airport2 is:  0
Total schedule delay is:  696
Max shift:  6
Number of unaccommodated requests:  0

---------------------------------------------------------

Initial Number of unaccommodated requests:  376
Initial count of unit violation for airport1 is:  33
Initial count of unit violation for airport2 is:  26
Final count of unit violation for airport1 is:  0
Fina

In [9]:
env = TwoAirportSchedEnv(number_of_actions=13, number_of_requests=1000, num_airports=2, cap_per_airport_arr=[6,6])
model = DQN.load("dqn_schedenv_full_setting2")

def evaluate_agent(scenarios):
    for scenario in scenarios:
        # Set the environment to the current scenario
        obs, info = env.set_scenario(scenario)
        counter = 0
        print('Initial Number of unaccommodated requests: ', env._unaccom_req)

        while True:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)

            if done or counter == 5000:
                env.evaluate_metrics()
                print("\n---------------------------------------------------------\n")
                break

            counter += 1

evaluate_agent(scenarios)

Initial Number of unaccommodated requests:  370
Initial count of unit violation for airport1 is:  31
Initial count of unit violation for airport2 is:  26
Final count of unit violation for airport1 is:  0
Final count of unit violation for airport2 is:  0
Total schedule delay is:  760
Max shift:  6
Number of unaccommodated requests:  0

---------------------------------------------------------

Initial Number of unaccommodated requests:  313
Initial count of unit violation for airport1 is:  25
Initial count of unit violation for airport2 is:  22
Final count of unit violation for airport1 is:  0
Final count of unit violation for airport2 is:  0
Total schedule delay is:  647
Max shift:  6
Number of unaccommodated requests:  0

---------------------------------------------------------

Initial Number of unaccommodated requests:  376
Initial count of unit violation for airport1 is:  33
Initial count of unit violation for airport2 is:  26
Final count of unit violation for airport1 is:  0
Fina

In [14]:
# import gym

# from stable_baselines3 import DQN

# env = TwoAirportSchedEnv(number_of_actions=num_actions, number_of_requests=1000, num_airports=2, cap_per_airport_arr=[6,6])

# model = DQN.load("dqn_schedenv_full_setting3")

# def evaluate_agent(n_episodes=20):
#     obs, info = env.reset()
        
#     for _ in range(n_episodes):
#         counter = 0
        
#         while True:
#             action, _states = model.predict(obs, deterministic=True)
#             obs, reward, done, truncated, info = env.step(action)
    
#             if done or counter == 5000:
#                 env.evaluate_metrics()
#                 print("\n---------------------------------------------------------\n")
#                 obs, info = env.reset()
#                 break
                
#             counter += 1
            
# evaluate_agent()

env = TwoAirportSchedEnv(number_of_actions=13, number_of_requests=1000, num_airports=2, cap_per_airport_arr=[6,6])
model = DQN.load("dqn_schedenv_full_setting3")

def evaluate_agent(scenarios):
    for scenario in scenarios:
        # Set the environment to the current scenario
        obs, info = env.set_scenario(scenario)
        counter = 0
        print('Initial Number of unaccommodated requests: ', env._unaccom_req)

        while True:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)

            if done or counter == 5000:
                env.evaluate_metrics()
                print("\n---------------------------------------------------------\n")
                break

            counter += 1

evaluate_agent(scenarios)

Initial Number of unaccommodated requests:  370
Initial count of unit violation for airport1 is:  31
Initial count of unit violation for airport2 is:  26
Final count of unit violation for airport1 is:  29
Final count of unit violation for airport2 is:  25
Total schedule delay is:  15
Max shift:  5
Number of unaccommodated requests:  356

---------------------------------------------------------

Initial Number of unaccommodated requests:  313
Initial count of unit violation for airport1 is:  25
Initial count of unit violation for airport2 is:  22
Final count of unit violation for airport1 is:  25
Final count of unit violation for airport2 is:  22
Total schedule delay is:  5
Max shift:  5
Number of unaccommodated requests:  313

---------------------------------------------------------

Initial Number of unaccommodated requests:  376
Initial count of unit violation for airport1 is:  33
Initial count of unit violation for airport2 is:  26
Final count of unit violation for airport1 is:  3

In [17]:
env = TwoAirportSchedEnv(number_of_actions=13, number_of_requests=1000, num_airports=2, cap_per_airport_arr=[6,6])
model = DQN.load("dqn_schedenv_full_setting4")

def evaluate_agent(scenarios):
    for scenario in scenarios:
        # Set the environment to the current scenario
        obs, info = env.set_scenario(scenario)
        counter = 0
        print('Initial Number of unaccommodated requests: ', env._unaccom_req)

        while True:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)

            if done or counter == 5000:
                env.evaluate_metrics()
                print("\n---------------------------------------------------------\n")
                break

            counter += 1

evaluate_agent(scenarios)

Initial Number of unaccommodated requests:  370
Initial count of unit violation for airport1 is:  31
Initial count of unit violation for airport2 is:  26
Final count of unit violation for airport1 is:  33
Final count of unit violation for airport2 is:  24
Total schedule delay is:  1715
Max shift:  49
Number of unaccommodated requests:  263

---------------------------------------------------------

Initial Number of unaccommodated requests:  313
Initial count of unit violation for airport1 is:  25
Initial count of unit violation for airport2 is:  22
Final count of unit violation for airport1 is:  20
Final count of unit violation for airport2 is:  19
Total schedule delay is:  1479
Max shift:  36
Number of unaccommodated requests:  214

---------------------------------------------------------

Initial Number of unaccommodated requests:  376
Initial count of unit violation for airport1 is:  33
Initial count of unit violation for airport2 is:  26
Final count of unit violation for airport1